# Chart Analysis Functions

In [14]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from spotifyData import spotifyData
from chartArtistAlbumData import chartArtistAlbumData
from chartUtils import *
from extraArtists import extraKnownArtists


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-02-15 15:40:26.610861


# Rename Input Data

In [ ]:
updateMultiDB(2)

In [2]:
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)

def updateMultiDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 2:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.save()
        
        multimanDB = masterArtistNameDB("multi", init=False)
        saveFile(idata=multimanDB.getRenames(), ifile="multi.yaml")

def updateManDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.checkForRecursives()

    if step == 2:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    if step == 3:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 4:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    
    _, _ = clock("Last Ran")

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7519 artist keys.
  There are currently 8788 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 147 artist keys.
  There are currently 153 renamed artist keys.


In [ ]:
updateMultiDB(2)

# Reverse Chart Lookup

In [ ]:
updateMultiDB(1)

In [ ]:
updateMultiDB(2)

In [ ]:
updateManDB(1)

In [ ]:
updateManDB(2)

In [ ]:
if False:
    from multiArtist import multiartist
    mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
    mularts.setKnownMultiDelimArtists(getFile("../multiartist/knownMultiArtists.yaml"))
    knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
    print(len(knownMultiArtists),"Known Artists")

    ignoresComp = ['Soundtrack', 'Various Artists', 'Original Broadway Cast Recording', 'Read-Along','Veggietales', 'Pokemon', 
                   'Barney', 'Walt Disney Read-Along', 'The Powerpuff Girls',
                   'Superstar Kidz', 'Original Cast', 'Original Cast Recording']
    ignores = ignoresComp    

    def matchArtist(artistName):
        subNames = mularts.getArtistNames(artistName)
        result = {subName: mdbmap.isKnownByName(subName) for subName in subNames}
        for subName in result.keys():
            if subName in ignores:
                result[subName] = True
        return result


    for chartName in chartDFs.keys():
        try:
            chartDFs[chartName]['DB'] = chartDFs[chartName]['RenamedArtist'].apply(matchArtist)
        except:
            print("Error with {0}".format(chartName))

In [ ]:
vals = {}
for chartName in chartDFs.keys():
    try:
        vals[chartName] = chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))].shape[0]
    except:
        pass
from pandas import Series

In [ ]:
Series(vals).sort_values(ascending=False)

In [ ]:
mdbmap.isKnownByName("Salt-N-Pepa")

In [ ]:
chartDF = chartDFs["vinyl-albums"]
def getMissingArtists(chartDF):
    missing = chartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]
    return list(set(getFlatList(missing["DB"].apply(lambda x: [k for k,v in x.items() if v is False]))))
getMissingArtists(chartDF)

In [ ]:
def getMissingArtists(chartDF):
    ichartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]

In [ ]:
chartName = "independent-albums"
chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))]

In [ ]:
bMap = {}
for chartTitle,chartName in initMap.items():
    newName = newMap.get(chartTitle)
    if newName is not None:
        bMap[newName] = chartName


In [ ]:
saveFile(ifile="billboardMapping.p", idata=bMap)

In [ ]:
bMap = getFile("billboardMapping.p")

# Top40 Data

In [ ]:
%load_ext autoreload
%autoreload
from top40Data import top40Data
td = top40Data(minYear=1, maxYear=2021)
td.setChartUsage(rank=[0,1,2,3,4,5,6])
td.setDBRenames(manDB)
td.setMultiDBRenames(multimanDB)
td.setFullChartData()
td.setArtistAlbumData()
td.saveArtistAlbumData()
td.saveFullChartData()

_, _ = clock("Last Run")

# Billboard Data

In [ ]:
%load_ext autoreload
%autoreload
from billboardData import billboardData
bd = billboardData(minYear=1, maxYear=2021)
bd.setChartUsage(rank=[0,1,2,3,4,5,6,7,8])
bd.setDBRenames(manDB)
bd.setMultiDBRenames(multimanDB)
bd.setFullChartData()
bd.setArtistAlbumData()

bd.saveArtistAlbumData()
bd.saveFullChartData()

_, _ = clock("Last Run")

# BillboardYE Data

In [ ]:
%load_ext autoreload
%autoreload
from billboardYE import billboardYE
bYE = billboardYE(minYear=1, maxYear=2021)
bYE.setChartUsage(rank=[0,1,2,3,4,5,6,7]) #,6,7])
bYE.setDBRenames(manDB)
bYE.setFullChartData()
bYE.setArtistAlbumData()

bYE.saveFullChartData()
bYE.saveArtistAlbumData()

_, _ = clock("Last Run")

# MusicVF

In [ ]:
%load_ext autoreload
%autoreload
from musicVFData import musicVFData
mvf = musicVFData(minYear=1, maxYear=2021)
mvf.setDBRenames(manDB)
mvf.setMultiDBRenames(multimanDB)
mvf.setChartUsage(rank=[0])
mvf.setFullChartData()
mvf.setArtistAlbumData()
mvf.saveArtistAlbumData()
mvf.saveFullChartData()
_, _ = clock("Last Run")

# Spotify Code

In [ ]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()

In [197]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[13]) #,1,2,3,4,5,7,8,9,10,11,12])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Charts For Rank 13
    Categories: ['world']
	Chart: world
  Using 1 Charts
Using 1 Charts For Rank 13
=== ChartUsage ===
  Using Charts (Rank=[13]): ['regional-global-weekly']
Found 68 summary files
  Using regional-global-weekly
Renamed 972 single artists
Saving 409 Artist Album Data to currentSpotifyArtistAlbumData.p
Saving data to currentSpotifyArtistAlbumData.p
  --> This file is 23.1kB.
Saved data to currentSpotifyArtistAlbumData.p
  --> This file is 23.1kB.
Saving 409 Full Artist Data
Saving data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 34.6kB.
Saved data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 34.6kB.
Current Time is Tue Feb 16, 2021 12:58:04 for Last Run


In [ ]:
updateManDB(2)

# Load MainDB

In [4]:
recreate = False
if recreate:
    maindb = mainDB(mdb=None, create=True, debug=True)
    maindb.setDBArtists(recreate=False)
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()
else:
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()    
_, _ = clock("Last Run")

=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rateyourmusic   =========================
=========================   deezer   =========================
=========================   albumoftheyear   =========================
=========================   genius   =========================
=========================   kworbspotify   =========================
=========================   kworbyoutube   =========================
=========================   kworbitunes   =========================
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 773808 ID -> Name entries
    Found 703262 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =======

# Analyze Data

In [198]:
## Basic stuff
%load_ext autoreload
%autoreload
from chartArtistAlbumData import chartArtistAlbumData


singleArtistAlbumData = {}
manyArtistAlbumData   = {}

chartType = "Spotify"
mType     = "Full"
cad = chartArtistAlbumData(chartType, ignoreMultiNames=False)
cad.createIndivArtistAlbumData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
chartArtistAlbumData(Spotify)
Found 62 files.
Adding 621 known multi delim artists.
  Assigning 621 known multi-name artists
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7519 artist keys.
  There are currently 8788 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 147 artist keys.
  There are currently 153 renamed artist keys.
There are 409 unique artists in artist albums data
There are 409 newly unique artists in artist albums data
Renamed 0 artists
Renamed 0 artists (multi)
There are 409    unique artist entries in the artist albums data
There are 407    single artist entries in the artist albums data
There ar

# Create Master Match Class

In [199]:
def showMatchedStatus(chartType, mdbmaps, mdbmc):
    print("Matched {0} / {1} Artists for {2} ChartType".format(mdbmaps[chartType].getSize(), len(mdbmc.matchData[chartType]), chartType))
    
def showRemainingMatches(chartType, mdbmc):
    print('='*100)
    print("="*10,"Artists That Were Not Tested (Low Albums)",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
    print("="*10,"Previously Tested Artists Without A Match",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    print('='*100)    
    
def analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=0.1):
    dbRenames = {}
    toget   = {}
    togetID = {}
    for primaryKey,artistName in mdbmaps[chartType].getArtists().items():
        primaryArtistName = artistName
        artistData = mdbmaps[chartType].getArtistDataByKey(primaryKey)
        for db,dbID in artistData.getDict().items():
            if db in ["DatPiff", "MetalStorm", "CDandLP", "AceBootlegs", "RockCorner", "MusicStack"]:
                continue
            if dbID is not None:
                secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
                if secondaryArtistName is None:
                    if toget.get(db) is None:
                        toget[db] = []
                        togetID[db] = {}
                    toget[db].append(primaryArtistName)
                    togetID[db][dbID] = primaryArtistName
                    continue
                s = SequenceMatcher(None, primaryArtistName, secondaryArtistName)
                ratio = s.ratio()
                maxRatio = maxVal
                if ratio >= maxRatio:
                    continue

                if ratio >= maxRatio-diffVal:
                    if dbRenames.get(secondaryArtistName) is not None:
                        if dbRenames[secondaryArtistName] == primaryArtistName:
                            continue
                        else:
                            pass
                    dbRenames[secondaryArtistName] = primaryArtistName
                    print("[{0: <30} {1: <4} {2: >30}] \t --> ({3}) {4} / {5} ({6})".format(primaryArtistName,round(ratio,2),secondaryArtistName,chartType,primaryKey,dbID,db))


    print("Found {0} artists to get.".format(len(toget)))
    print("Found {0} artists to rename.".format(len(dbRenames)))
    return toget, togetID, dbRenames    
    
def removeDBRenames(mdbmaps, chartType, dbRenames):
    for k,v in dbRenames.items():
        kKey = mdbmaps[chartType].getPrimaryKey(artistName=k, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)

        kKey = mdbmaps[chartType].getPrimaryKey(artistName=v, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)
    mdbmaps[chartType].save()
    
    
def analyzeRenames(manDB, dbRenames):
    redos = {}
    dels = []
    if len(dbRenames) == 0:
        print("Nothing to process.")
        return
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys() and bestname not in manDB.artistNameDB.keys():
            print(oldername,bestname)

            redos[bestname] = oldername
            dels.append(oldername)
            print('\t',oldername,'\t',bestname)

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))
    for oldername in dels:
        del dbRenames[oldername]

    print("# Redos: {0}".format(len(redos)))
    dbRenames.update(redos)
    print("# Renames: {0}".format(len(dbRenames)))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")







    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")
    #print(len(dbRenames))
    
    
        
    
def manualAppends(toMatch, chartType, minAlbums=0, add=True):
    if len(toMatch[chartType]) == 0:
        print("Nothing to append...")
        return
    
    toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
    toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
    toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)
    
    for idx,row in toMatchDF.iterrows():
        if add:
            artistName = row["ArtistName"]
            albumNames = row["ArtistAlbums"]
            nAlbums    = row["NumAlbums"]
            
            if nAlbums < minAlbums:
                continue
            print("add{0}(mdbmaps, {1}, {2}, {3})   ### [{4}]".format("AllMusic", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            print("add{0}(mdbmaps, {1}, {2}, {3})    ### [{4}]".format("Deezer", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            print("### {0}".format("  ,  ".join(albumNames)))
            for albumName in albumNames:
                print("### {0}  ---> [{1}]".format(albumName, cad.getAlbumsData(albumName)))
            print("\n")
        else:
            print("ignores.append(\"{0}\")".format(artistName))
            if " & " in artistName:
                continue
                print("ignores.append(\"{0}\")".format(artistName))

    print("mdbmaps[{0}].save()".format("\"{0}\"".format(chartType)))    
    





        
def copyMapData(mdbmap):
    return
    mdbmap.saveCopy()

def saveMapData(mdbmap, result_list):
    for item in result_list:
        primaryKey = item[0] 
        artistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        mdbmap.addArtistByKey(primaryKey, artistName=artistName, artistID=artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistDataByKey(primaryKey, db, matchID)

    mdbmap.save()
    

def matchItAll(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None):
    if toMatch is None:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=10000, maxAlbums=50000)

    if len(toMatch) == 0:
        return

    num_processes = 3
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]
    copyMapData(mdbmaps[dbName])

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)


    start, cmt = clock("Saving...")
    saveMapData(mdbmaps[dbName], result_list)
    elapsed(start, cmt)
    print("\nSleeping for 10 seconds...\n")
    #sleep(10)
    
    #mdbmc.matchMutualMaps()

    
def matchDBArtistWithoutAlbums(item, *args, **kwargs):
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = None
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score']),
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm

# MDB Maps

In [200]:
mdbmaps = {}
print("ChartType = {0}".format(chartType))
print("MType     = {0}".format(mType))
mdbmaps[chartType] = musicDBMap(chartType, init=False, copy=False)
mdbmc = masterDBMatchClass(maindb, mdbmaps)

manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)


def reMatch(mdbmaps=mdbmaps, mdbmc=mdbmc, mtype="Single"):
    if mtype == "Single":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getSingleArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Many":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getManyArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Full":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getFullArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
        
reMatch(mtype=mType)

ChartType = Spotify
MType     = Full
  Valid DBs: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm', 'Deezer', 'AppleMusic', 'AlbumOfTheYear', 'Genius', 'IHeart', 'KWorbSpotify', 'KWorbiTunes', 'KWorbYouTube']
  Loaded 12290 previously matched entries
Loading Artist Names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7519 artist keys.
  There are currently 8788 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 147 artist keys.
  There are currently 153 renamed artist keys.
  Setting matchData for Spotify


## Master Ignore List

In [201]:
ignoresList = ["chartIgnores.yaml"]
ignoresList = []

ignoresList = []
ignoresList.append("Artists Stand Up To Cancer")
ignoresList.append("Cast Of Rent")
ignoresList.append("2006 Broadway Cast Recording")
ignoresList.append("Original Broadway Cast Of Dear Evan Hansen")
ignoresList.append("Various")
ignoresList.append("Various Artists")
ignoresList.append("Read-Along")
ignoresList.append("Varios Artistas")
ignoresList.append("Artists Against Bullying")
ignoresList.append("Varios")
ignoresList.append("Sleep Music Lullabies")
ignoresList.append("Yoga Meditation Tribe")
ignoresList.append("The Cast Of Stomp")
ignoresList.append("Quincy Jones Feat. The Cast Of Stomp/The Yes/No Productions")
ignoresList.append("Walt Disney Read-Along")
ignoresList.append("Classical Lullabies For Babies Academy")
ignoresList.append("Diverse")
ignoresList.append("Blandade Artister")
saveFile(idata=ignoresList, ifile="chartIgnoreArtists.yaml")

unknownList = []
unknownList.append("Amil-lion")
unknownList.append("Yung Lito")
unknownList.append("Idols")
unknownList.append("X Factor Finalists")
unknownList.append("R")
unknownList.append("Malena")
unknownList.append("Neales")
unknownList.append("Incognito")
unknownList.append('"Patches" Stewart')
unknownList.append("X Factor NZ Top 12")
saveFile(idata=unknownList, ifile="chartUnknownArtists.yaml")

internationalList = []
internationalList.append("3robiu")
internationalList.append("Multi-interpretes")
internationalList.append("Mormon Tabernacle Choir/Orch. At Temple Square/Yoncheva/Mumford/Villazon/Terfel (Wilberg)")
internationalList.append("Villazon")
internationalList.append("Mumford")
internationalList.append("Vienna Philharmonic (Dudamel)")
internationalList.append("Louisville Orchestra (Abrams)")
internationalList += ['Eight Ranger', 'Yi Yang Qian Xi', "ONE N' ONLY", "MoLa", "Sus Invitados"]
internationalList += ['Juanma Y Su Tuna Para Todo El Ano', 'NYC', 'Nogizaka 48', 'Sho Kiryuin from Golden Bomber']
internationalList += ['The MONSTERS', 'ZERO-G', 'Huang Lige', 'Renzo', 'Bodan Shonen Dan', 'ST RISH', 'King Cream Soda']
internationalList += ['Good Morning America', 'Hotta Ke BAND', 'Feng Jianyu', 'Stanley Most']
internationalList += ['Cardiff City Fc', 'Leeds United Team', 'Monkey Hangerz']
internationalList += ['Good Child Foundation', 'Neon Brotherhood', 'Rogue Souljahz']
internationalList += ['GMC', 'Krossfade', 'Dave Howley', 'Toko', 'Y-not', 'Afro Music']
internationalList += ['Husvenne', 'Maca Boy', 'Skyndeep', 'TwitterXmassingle', 'Julieanne Dineen', 'Julie-Anne Dineen', 'Anybodies']
internationalList += ["Miro"] # Bulgaria
internationalList += ["Lama", 'Vintaj', 'D-flow', 'U. Dumanska', 'Leonard', 'MARY', 'Naomi', 'IKE', 'FIESTA!', 'Francisco Loría']
internationalList += ['SANARI', 'KIRA', 'Deep Ng', 'BOYZ', 'SEUNGRI', 'TLDK', 'Della', 'Dadado Huang', 'WONFU', 'DEN', 'PORTRAIT', '2T FLOW', 'GULF', 'VANGOE', 'DA 阿達']
internationalList += ['Renan Almendarez Coello', 'Alfredo Ramirez Corral']
internationalList += ['K-One', 'K.One', 'Miss Murphy', 'Bandits', 'Pop', 'Industry', 'Bigg Face', 'Atlas', 'Chico', 'Pictures', 'Jay-Jay Harvey']
internationalList += ['Paw Justice', 'Jay-Jay Feeney', 'Guy Williams & Scribe', 'Dominic Harvey & Scribe']
internationalList += ['Ole']

saveFile(idata=internationalList, ifile="chartInternationalArtists.yaml")

tonewUnknownList  = ['Lil Adrei', 'The 7 Dimensions', 'HIXTAPE', 'Porky Slim']
tonewUnknownList += ['SL', 'Sidemen', 'Mastermind', 'HOSH', 'Frosty']
saveFile(idata=tonewUnknownList, ifile="chartToNewArtists.yaml")

{'Gabo Parisi':9846490}
{'Rory & The Island': 1219224, 'Sean Og': 537158, "Hoo Leeger": 10908080}
{'The Suspects and Guests': 5061058, "Dennis Reed & Gap": 15080705}
{}

{"Eri Esittajia": 243610}

soundtrackList = ['Jul På Vesterbro', 'Troy & Gabriella']
saveFile(idata=soundtrackList, ifile="chartSoundtrackArtists.yaml")

applemusicList  = ['Dutch','Deno','DHT','Sadbh']
applemusicList += ['DDG', 'Lean Trap', 'MOLO', 'Amina']
applemusicList += ['Rocket Girls 101', "David Miles", "Cyril"]
applemusicList += ["Camur"]
saveFile(idata=applemusicList, ifile="chartAppleMusicArtists.yaml")

ignoresList = ["chartIgnoreArtists.yaml", "chartUnknownArtists.yaml", "chartInternationalArtists.yaml",
               "chartToNewArtists.yaml", "chartAppleMusicArtists.yaml", "chartSoundtrackArtists.yaml"]


## Match Using Names And Albums

In [202]:
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
reMatch(mtype=mType)
print("Done.\n\n")

========== 50
========== 20
========== 10
========== 5
========== 3
========== 2
========== 1
  Setting matchData for Spotify
Done.




In [203]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Matched 12290 / 411 Artists for Spotify ChartType
========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========


## Match Only Using Names (Artists Without A Previous Match)

In [204]:
for i in range(1):
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    if len(toMatch[chartType]) == 0:
        break
    matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

  Setting matchData for Spotify


In [205]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Matched 12290 / 411 Artists for Spotify ChartType
========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========


## Try To Match Artists That Were Not Previously Tested (Low Albums)

In [206]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

  Setting matchData for Spotify


In [207]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Matched 12290 / 411 Artists for Spotify ChartType
========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========


## Extra Artists Info

In [208]:
#from extraArtists import extraKnownArtists
#extraKnownArtists(mdbmaps, chartType)
#showMatchedStatus(chartType, mdbmaps, mdbmc)
#showRemainingMatches(chartType, mdbmc)

#### Unmatched Artists

In [209]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, 
                                 minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
len(toMatch[chartType])

0

# Manual Appends

In [36]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Joshua Bassett', 'DJ Scheme', 'Pedro CapÃ³', '16yrold', 'Sueco', 'BRELAND', 'Alicia Moffet', 'Skye', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'VIC MENSA', 'NEW CITY', 'Madalen Duke', 'Cassow', 'DiRTY RADiO', 'Bob Pressner', 'Christian Hudson', 'Vanic', 'Elijah Woods', 'Jamie Fine']


In [50]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Mimi Webb', 'Frisco', 'Morrisson', 'MizOrMac', 'Miniminter', 'Fekky', 'Kenny Allstar', 'Rv', 'TiÃ«sto', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'OSH', 'Koomz', 'Amelia Monét', 'Tranell', 'Alex Ross', 'Ralph Felix', 'David Campbell', 'Vidwaan Boon', 'Lea Heart', 'Ché-Fu', 'Lee Mvtthews', 'SACHI', 'Elektriches Weihnachtfest', 'Trill Ryan', 'TEEKS', 'Maimoa', 'J1', 'The Blue Jeans', 'Bob B. Soxx', 'Collectors', 'Henny', 'Zinny', 'AP']


In [65]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['DJ JEEZY', 'Maya Berovic', 'Neptunica', 'Poptracker', 'Zappi', 'LZ27', 'Milan Stankovic', 'Standart Skill', 'Butrint Imeri', 'Tanja Savic', 'Monja Mari Achi', 'Die Hollerstauden', 'Claudius Vlasak', 'DJ Selecta', 'Klaus Peter', 'Ellis Adventure', 'Mortal Frame', 'DJ Greg V', 'Mister Claude', 'Little Strong', 'River Tyne', '4 Giants', 'Martina Kaiser', 'Patrick Knight', 'A Stritzi', 'Shlump', 'BENGELS', 'Stefanie Kloß', 'ARTY', 'Nullzweizwei', 'Koushino', '65Goonz', 'Pietro Basile', 'Sadri', '2ara', 'Provinz', 'Laas Unltd.', 'Z', 'HAMZO 500', 'Gewitter im Kopf', 'Erabi', 'Koosen', 'O.G.', 'Ramon Roselly', 'Senna Gammour', 'Maaf', 'Riccardo', 'ToTheMoon', 'Teddy Teclebrhan', 'Sipo', 'Joker Bratan', 'Bato', 'Dafina Zeqiri', 'Oflow', 'Dj Iljano', 'OFFTR3ND', 'F4ilure', 'Jasin Blanco', 'Pronto', 'Rocio Alvarez', 'Swiluu', 'Mimiks', 'LCone', 'Zwei Am Morge', 'Cricket', 'LIBA', 'Manillio', 'SHINSKY', 'Pappa Razzi', 'Carnie', 'Lily']


In [85]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Enula', 'Raffaele Renda', 'Aka 7even', 'Marïna', 'WØODY', 'Niko Pandetta', 'Cancun', 'Milano Ovest', 'Axos', 'Marta Daddato', 'Tosca', 'MARCO SENTIERI', 'Paolo Jannacci', 'Gabriella Martinelli', 'NOËP', 'Chakra', 'Greta Menchi', 'Rosalba', 'Sherol Dos Santos', 'Renza Castelli', 'Zano', 'Giovanni Caccamo', 'Lorenzo Baglioni', 'Ron', 'Mirkoeilcane', 'Renzo Rubino', 'Lorenzo Licitra', 'Andrea Radice', 'ROS', 'Raige', 'Maldestro', 'Francesco Guasti', 'Lesram', 'Bilton', 'joysad', 'Dioscures', 'Kanoé', 'GS', 'ISK', 'Osen', 'T Garcia', 'twinsmatic', 'DENZO', 'Kodes', 'Zikxo', 'Badjer', '404Billy', 'Kopp Johnson', 'Sirius', 'Seezy', 'Jahyanai King', 'Jaded', 'Nassi', 'DJ JEEZY', 'Wahlstedt', 'Don Criminals', 'Novi', '5HOW', 'Wikw', 'Goonew', 'Edo Ferragamo', 'Lost Identities', 'GioBulla', 'Roxx Remora', 'Otira', 'R3LL', 'Brooze', 'DASIC', 'DJ LeanGun', 'Toni der Assi', 'Maale Gars', 'Nevlo', 'Nosi', 'Chasey Negro', 'The FifthGuys', 'KOLIDESCOPES', 'Hanzy', 'thisisNAMASTE', 'Jared Moreno', '

In [101]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Whyrun', 'Villi Neto', 'Gummi Tóta', 'Unnsteinn', 'Miquela', 'Ian Erix', 'Chris Tyson', 'Greifarnir', 'Magni', 'Vilhjálmur Vilhjálmss', 'Hreimur', 'Venjulegir Gaurar', 'Sigga Beinteins', 'Edda Heiðrún Backman', 'Eythor Ingi', 'VICTORIA', 'Ferðumst innanhúss', 'Ouse', 'R3SYNTH', 'fuse.', 'Scraby', 'Sharp Man', 'Br0med', 'krassasig', 'Háski', 'Andri Már', 'Shakespeare verður ástfanginn', 'Reynir', 'EVR3stNess', 'Kolassoid', 'Akeckee', 'ZENA', 'zalagasper', 'Eliot', 'Michael Rice', 'S!sters', 'Tulia', 'Srbuk', 'Jurijus', 'Sarah McTernan', 'Flosi V', 'Screamerboys', 'Egill Stolz', 'Helgi B', 'Stefán Karl Stefánsson', 'Bergur Leó', 'Caulfield', 'ZÖE', 'Youngcuzzoo', 'Óðinn Valdimarsson', "J'adora", 'Ieva Zasimauskaitė', 'Lea Sirk', 'DoReDoS', 'SuRie', 'Flosi', '$igmund', 'Snow Dept.', 'Ragga Holm', 'iDubbbz', 'Dihaj', 'Hovig', 'IMRI', 'Jana Burčeska', 'Slavko Kalezić', 'Rúnar Eff', 'Þórdís Birna Borgarsdóttir', 'Erna Mist Pétursdóttir', 'Rakel Pálsdóttir', 'Sara Farell', 'TheoFuego', 'Fra

In [117]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Carl Knast', 'DIBSET', 'SunCity', 'Boris Laursen', 'Jamie Kamara', 'Børnemusik Dyrene', 'Awave', 'N3LLY', 'HAM fra Syd', 'Merro8', 'IBO', 'Berd', 'LayLay', 'Jesu Bankerz', 'Kristian Kjærlund', 'Alexander Husum', 'AMRO', 'Leprom', 'Karizman', 'Elvira Pitzner', 'VM Holdet', 'KING', 'Vkation', 'Tramper Torben', 'Place On Earth', 'Delahoia', 'Sandra Hussein', 'Soleima', 'Tore', 'Tiril', 'Henrik Blomme', 'Katie Keller', 'Salazar', 'Dj Guldsmed', 'Jytte', 'Lakserytteren', 'CHILI', 'MGP Allstarz 2017', 'Guldpige', 'JB', 'Euroo', 'Marti', 'Malaka', 'buller', 'Oskar Häggström', 'A36', 'Seedy', 'HON', 'SG Lewis', 'Style', 'Erik Lundin', 'Matte Caliste', '2.clock', 'Abidaz', 'Biggie Juke', 'Suzi P', '2M', '10an', 'Linda Olsson', 'Bailey Jehl', 'Four of Diamonds', 'Janice', 'The Mayries', 'Jelassi', 'Wild-Boy', 'Michael Rice', 'zalagasper', 'Herbert Munkhammar', 'Jonathan Johansson', 'Mila Crowell', 'Lionman', 'Leyton Lake', 'Amy Soonyoung', 'Tony Lieberman', 'Hallman', 'Max George', 'Ieva Zasim

In [133]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Kamal Khaira', 'Madhu Priya', 'Roll Rida', 'Raghav Chaitanya', 'Simran Kaur Dhadli', 'Khushank Dalal', 'Nimrat Khaira', 'Sickflip', 'Kulbir Jhinjer', 'Nirvair Pannu', 'Bhuvan Bam', 'Korala Maan', 'Tanzeel Khan', 'Sam C.S.', 'The Landers', 'Sanjith Hegde', 'Arun Dev Yadav', 'Sarath Santosh', 'Amruta Fadnavis', 'Happy Raikoti', 'Mickey J. Meyer', 'Rupinn', 'Nitin Mukesh', 'Tamir Bar', 'Orr Amrami Brockman', 'Tay G', 'Jimbo J', 'Teddy Neguse', "100 First Songs' Participants", 'Francis Duran', 'Glen Dean', 'Føx & the Hound', 'Sergio', 'Pako Ramirez', 'Young Wavee', 'דקל וקנין', 'Berg', 'Roy Kafri', 'ZENA', 'Asaf Goren', 'Ishay Levi', 'Avior Malasa', 'Yossi Shitrit', 'Gher', 'Michael Swissa', 'אבי פאנל', 'Katrix and Doron Beaton', 'Liron Amram', 'Omri 69 Segal', 'Buttering', 'Afroto', 'Ahmed Santa', 'Mido Gad', 'Angham', 'Barry', 'Ali Adora', 'Abo Lila', 'Emad Kamal', 'Karim Mohsen', 'Rain Sleepings Sounds', 'Limit 29', 'Rain Sound Studios', 'Alex Ran', 'Barachi', 'Idahams', 'Rc $pitta', 

In [149]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Sergi', 'La Mosca Tse-Tse', 'Marcos Menchaca', 'Gera MX', 'Heroes Del Silencio', 'Yuawi', 'Где Фантом?', 'Kara Kross', 'SEEMEE', 'PALC', 'ElyOtto', 'Dzharakhov', 'LIDA', 'G4OUR', 'CAPTOWN', 'Smoky Mo', 'Pacosp', 'IsaacMusic', 'Gunwest', 'Ямаджи', 'Armando Hernandez', 'Ryan Roy', 'Leebrian', 'DJ Goozo', 'LosPetitFellas', 'Dejota2021', 'Luisa Fernanda W', 'Koffee el Kafetero', 'Kenai', 'Sanluis', 'Martin Elías', 'Leo Mattioli', 'Tambó Tambó', 'Felix Gabriel', 'DJ Yayo', 'La Nueva Luna', 'Wanakin', 'Joleane', 'Joe Hasin', 'Miz Lina', 'Nadia Dan', 'Aida gean', 'Karl Michelson', 'Miles Kenn', 'Kennis Inn', 'Nickl Mons', 'Adam Dank', 'Justin Tomber', 'Edik', 'Dean Loo', 'Vitya AK', 'payton', 'Mitchel', 'Фикс', 'Gocata Kangal', 'VessoU', 'Togasang', 'Havenbled', 'Jamaicataxi7', "Leron Rankin'", 'Gcollectah', 'Crystal Byrd', 'Donika', 'Riko Band', 'LZ27', 'KNUX', 'MacShawn100', 'Nikki Luchese', 'FlameShadow', 'SayReal', 'NEEL', 'Leol', 'CashnOut Esco', 'StzCorp', 'VICTORIA', 'Ran Nishihara',

In [164]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['04 Limited Sazabys', 'SPiCYSOL', 'hachi', 'ACE COLLECTION', '稲村オーケストラ', 'PRODUCE 101 JAPAN', 'ヒプノシスマイク -A.R.B- (Division All Stars)', 'Whiteberry', 'CHIAKI SATO', 'AYA A.K.A PANDA', 'THE ORAL CIGARETTES', '浜田雅功と槇原敬之', 'Hypnosis Mic -D.R.B.- (MAD TRIGGER CREW)', 'Kinokoteikoku', "The Urashima's", 'Fumiya Fujii', 'Mone Kamishiraishi', 'Takehara Pistol', 'Ovall', 'Straightener', 'DOBERMAN INFINITY', '吳林峰', 'Panther Chan', 'Cath Wong', 'Yan Ting', 'Edan 呂爵安', 'my little airport', '石山街', 'Adason Lo', 'Thomas DGX YHL', '癲佬 dlo', 'Akano', 'YNW Jordan', 'Floating in Space - Project', 'Big Loso', 'Windy Guai', 'DNA', 'PBE PLUTO', 'Jt el Utility', 'Damián V', 'Sirvan Khosravi', '井谷俊二', 'MariKuroso', 'Mukaybin Mukayze', 'King Gunna', 'Zoee8', 'Bogart Bonales', 'Dennis Fernando', 'Friday Santana', 'Harj Savage', 'bruno livesax', 'JSnake', 'Keyzs-Sean Sparsfa', 'From Under Concrete Kings', 'ED LIIT', 'FAO>dino', 'Buunkin', 'Los Necesarios', 'TG', 'Phantom Caine', 'Molina Molina', 'Thee JAE', 'Kii

In [179]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['DJ Stokie', 'Bok Van Blerk', 'Dj Obza', 'Blaqnick', 'Emtee', 'DJ Jaivane', 'Vhudie', 'Mthunzi', 'Loufi', 'ANATII', 'K.O', 'David Scott', 'ILuvJupiter', 'Busiswa', 'GeeSixFive', 'Costa Titch', "Soweto's Finest", 'Yanga Chief', 'Shane Eagle', 'Nija', 'Stillaround', 'Salatiel', 'Gemini Major', 'Midnight Meetings', 'ABS-CBN Music All Star', 'Jana Garcia', 'Brian Pepito', 'Geo Ong', 'Reese Lansangan', 'Amber Leigh Irish', 'Andrea Babierra', 'Kiana V', 'Hatemost', 'ItsOnlySkillz', 'TiÃ«sto', 'Ofili', 'Popayedi', 'Vercetti', 'Natassa Bofiliou', 'Ripen', 'Ethismos', 'Alcatrash', 'Konnie Metaxa', 'Young Tulio$', '12os Pithikos', 'Rio', 'Athliens', 'Kanon', 'Arva', 'Pournaras', 'Eaze', 'Natasha Kay', 'Smuggler', 'Joker Two-Face', 'ZENA', 'Eisvoleas', 'Efta', 'Panos Mouzourakis', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'Kindler Shah', 'Trifecta', 'JJ Hannon', 'Vangelis Kakouriotis', 'LYSA', 'Hovig', 'Dihaj', 'Madstreet', 'Khifnu', 'CHA NI (SF9)', 'Dhika Fawaz', 'YUJU', 'Keisya Levronka', 'Canna

In [195]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

['Biu do Piseiro', 'Nêgo Jhá', 'OIK', 'Camila Loures', 'DJ Pernambuco', 'Anne', 'Rafael Allmark', 'Maycon e Vinicius', 'Thiago Carvalho', 'Fernanda Salgado', 'Thascya', 'Gabriel Elias', 'Furacao Love', 'Seakret', 'Junior Lord', 'Yuri NR5', 'Greg Ferreira', 'Fumaxa', 'MOBBERS', 'Gama WNTD', 'Yung SpaceGhost', 'WUANT', 'xtinto', 'SYRO', 'Instinto 26', 'Kappa Jotta', 'WiNDOH', 'Aragão', 'Mizzy Miles', 'Gson', 'Timor YSF', 'Achero', "Dino d'Santiago", 'Lhast', 'LON3R JOHNY', 'Irina Barros', 'Minguito 283', 'Dream Boyz', 'Betsy David', 'Waze', 'Valas', 'Domi', 'Mastiksoul', 'João Sousa', 'Miguel Alves', 'Apollo G', 'ÁTOA', 'sora9k', 'Joint One', 'Vado Más Ki Ás', 'Chris Tamayo', 'ARTIGAZZ', 'Jvst Fly', 'GROGNation', 'Mota JR', 'YUZI', 'prettieboy johnson', 'Huzz', 'Ray Polanco', 'Sea', 'Kazzio', 'Ella Nor', 'ANDREZO', 'Mundo Segundo', 'Zara G', 'April Ivy', 'Wuant', 'Carlão', 'HMB', 'Yasmine', 'Ray', 'Maria Cecília', 'Xutos', 'Pontapés']


In [ ]:
print([item[1]["ArtistName"] for item in toMatch[chartType]])

In [196]:
set1  = set(['Joshua Bassett', 'DJ Scheme', 'Pedro CapÃ³', '16yrold', 'Sueco', 'BRELAND', 'Alicia Moffet', 'Skye', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'VIC MENSA', 'NEW CITY', 'Madalen Duke', 'Cassow', 'DiRTY RADiO', 'Bob Pressner', 'Christian Hudson', 'Vanic', 'Elijah Woods', 'Jamie Fine'])
set2  = set(['Mimi Webb', 'Frisco', 'Morrisson', 'MizOrMac', 'Miniminter', 'Fekky', 'Kenny Allstar', 'Rv', 'TiÃ«sto', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'OSH', 'Koomz', 'Amelia Monét', 'Tranell', 'Alex Ross', 'Ralph Felix', 'David Campbell', 'Vidwaan Boon', 'Lea Heart', 'Ché-Fu', 'Lee Mvtthews', 'SACHI', 'Elektriches Weihnachtfest', 'Trill Ryan', 'TEEKS', 'Maimoa', 'J1', 'The Blue Jeans', 'Bob B. Soxx', 'Collectors', 'Henny', 'Zinny', 'AP'])
set3  = set(['DJ JEEZY', 'Maya Berovic', 'Neptunica', 'Poptracker', 'Zappi', 'LZ27', 'Milan Stankovic', 'Standart Skill', 'Butrint Imeri', 'Tanja Savic', 'Monja Mari Achi', 'Die Hollerstauden', 'Claudius Vlasak', 'DJ Selecta', 'Klaus Peter', 'Ellis Adventure', 'Mortal Frame', 'DJ Greg V', 'Mister Claude', 'Little Strong', 'River Tyne', '4 Giants', 'Martina Kaiser', 'Patrick Knight', 'A Stritzi', 'Shlump', 'BENGELS', 'Stefanie Kloß', 'ARTY', 'Nullzweizwei', 'Koushino', '65Goonz', 'Pietro Basile', 'Sadri', '2ara', 'Provinz', 'Laas Unltd.', 'Z', 'HAMZO 500', 'Gewitter im Kopf', 'Erabi', 'Koosen', 'O.G.', 'Ramon Roselly', 'Senna Gammour', 'Maaf', 'Riccardo', 'ToTheMoon', 'Teddy Teclebrhan', 'Sipo', 'Joker Bratan', 'Bato', 'Dafina Zeqiri', 'Oflow', 'Dj Iljano', 'OFFTR3ND', 'F4ilure', 'Jasin Blanco', 'Pronto', 'Rocio Alvarez', 'Swiluu', 'Mimiks', 'LCone', 'Zwei Am Morge', 'Cricket', 'LIBA', 'Manillio', 'SHINSKY', 'Pappa Razzi', 'Carnie', 'Lily'])
set4  = set(['Enula', 'Raffaele Renda', 'Aka 7even', 'Marïna', 'WØODY', 'Niko Pandetta', 'Cancun', 'Milano Ovest', 'Axos', 'Marta Daddato', 'Tosca', 'MARCO SENTIERI', 'Paolo Jannacci', 'Gabriella Martinelli', 'NOËP', 'Chakra', 'Greta Menchi', 'Rosalba', 'Sherol Dos Santos', 'Renza Castelli', 'Zano', 'Giovanni Caccamo', 'Lorenzo Baglioni', 'Ron', 'Mirkoeilcane', 'Renzo Rubino', 'Lorenzo Licitra', 'Andrea Radice', 'ROS', 'Raige', 'Maldestro', 'Francesco Guasti', 'Lesram', 'Bilton', 'joysad', 'Dioscures', 'Kanoé', 'GS', 'ISK', 'Osen', 'T Garcia', 'twinsmatic', 'DENZO', 'Kodes', 'Zikxo', 'Badjer', '404Billy', 'Kopp Johnson', 'Sirius', 'Seezy', 'Jahyanai King', 'Jaded', 'Nassi', 'DJ JEEZY', 'Wahlstedt', 'Don Criminals', 'Novi', '5HOW', 'Wikw', 'Goonew', 'Edo Ferragamo', 'Lost Identities', 'GioBulla', 'Roxx Remora', 'Otira', 'R3LL', 'Brooze', 'DASIC', 'DJ LeanGun', 'Toni der Assi', 'Maale Gars', 'Nevlo', 'Nosi', 'Chasey Negro', 'The FifthGuys', 'KOLIDESCOPES', 'Hanzy', 'thisisNAMASTE', 'Jared Moreno', 'Krozz', 'Boris Ayson', 'Alley', 'BROMYGO', 'Kroshwell', 'Nacho Fano', 'Kyd Knarly', 'EQRIC', '4Korners', 'Helen', 'KLOUD', 'Jason Thurell', 'Heuse', 'Lokyto', 'Clambake', 'Josh Stanley', 'Gazz', 'Brevin Kim', 'Dan Lemaire', 'Makasi', 'Mila Jam', 'Jeff Kush', 'EGalas', 'Uniekgrace', 'Basics', 'Camryn', 'Jay 305', 'NAZAAR', 'Benjin', 'VLS', 'Tom Berx', 'KNYD', 'Mark Melgo', 'EDGR', 'Neighbors', 'Jeyday', 'Flairo', 'FVR AVAY', 'Nicky Vaent', 'Vinni Wonka', 'Michael Wayne', 'Ricky Gazetta', 'Busta K', 'Prince Paris', 'Poorchoice', 'Blanke', 'Cytrax', 'Slue', 'Rex Stax', 'Brianna Mazzola', 'Meresha', 'GLDN', 'Simock', 'Ruben Pol', 'Those 2 Guys', 'Seorsia', 'Lennard Ellis', 'AXEC', 'SHANNI.', 'Sophia Treadway', 'Aysha Monet', 'Mystic Natives', 'Schuz', 'CRZY', 'Kalipzo', 'Dee Gomes', 'Lifetime Member', 'Tritan', 'teamwork.', 'The Unfits', 'The Him', 'Deadrose', 'JJ Hannon', 'Hones-T', 'Kyle Edwards;DJ Smallz 732', 'Kyle Edwards;Chris Bevrly', 'Miquela', 'Ian Erix', 'Chris Tyson', 'Dylan Diamond', 'Cal Scruby', 'Bandana', 'Lérica', 'Hard GZ', 'Aiman JR', 'Marc Seguí', 'Emanuel A', 'Yulien Oviedo', 'REYKO', 'FlowZeta', 'Joan Garrido', 'Damion Frost', 'Dave Zulueta', 'María Escarmiento', 'Pol Granch', 'Chica Sobresalto', 'Yung Sarria', 'Sebastián Cortés', 'TheGrefg', 'Paula Cendejas', 'Anaju', 'Eva B', 'Ankhal', 'Resistiré 2020', 'Samantha', 'Duo Dinamico', 'ARON', 'Javy Ramírez', 'Love Yi', 'Eklectico', 'Locoplaya', 'LUNA KI', 'Dylan Fuentes', 'Soge Culebra', 'Elilluminari', 'Sergio Ramos', 'Selecta', 'Kaze', 'Jirafa Rey', 'Paula Gonu', 'Yuawi', 'La Vida Moderna', 'Angeles', 'Tutto Duran', 'ICE', 'Dior', 'Goldkimono', 'Jeffrey Heesen', 'David Neres', 'Jjakoveertien', 'Rotjoch United', 'Wolfgang', 'Makkie', 'Joey Bravo', 'Jerra', 'Mario Cash', 'ValsBezig', 'Oualid', 'Team Rush Hour', 'DYSTINCT', 'Bigidagoe', 'KNALLER', 'Jordymone9', 'Pierrii', 'Matthy', 'Selma Omari', 'Frozen Fire', 'Akwasi', 'DJ Nazil', 'Pjotr', 'Trobi', 'Kevcody', 'Jayboogz', 'S10', 'The voice of Holland', 'Blow', 'Ares', 'Dv', 'Dylisa', 'Joel Beukers', 'Dana', 'DJ Wale', 'Souff', 'Floor Jansen', 'Paul Sinha', 'Lirical', 'Artistic Raw', 'LU BACH', 'Skiezo', 'Rene Karst', 'Jiri11', 'Nijo', 'MRD', 'Zoë-Jadha', 'Quatro Vision', 'Daniël Busser', 'F.I.', 'SuriBeats', 'Fabiola', 'Dave Roelvink', 'Iliass', 'Memphis Depay', 'Qucee', 'Shock Elias', 'Quessswho', 'RUGGED', 'Vonneke Bonneke', 'Scooby072', 'CJaye', '808milli', 'Bridde Boys', 'Lucass', 'Nesim Najih', 'Gebroeders Rossig', 'Anbu', 'Kid de Blits', 'Mael', 'Gamemeneer', 'Anu-D', 'BangBros', 'Milan Knol', 'Langesjaak', 'Darryl', 'Tavi Castro', 'Johnny Sellah', 'Minitrapper', 'Rabby Racks', 'RONDÉ', 'PLEUN', 'Angelo King', 'J1', 'Dean', 'Kellini', 'Fito', 'Waor', 'Natos', 'Prok', 'El High', 'Dayme', 'Diaz'])
set5  = set(['Whyrun', 'Villi Neto', 'Gummi Tóta', 'Unnsteinn', 'Miquela', 'Ian Erix', 'Chris Tyson', 'Greifarnir', 'Magni', 'Vilhjálmur Vilhjálmss', 'Hreimur', 'Venjulegir Gaurar', 'Sigga Beinteins', 'Edda Heiðrún Backman', 'Eythor Ingi', 'VICTORIA', 'Ferðumst innanhúss', 'Ouse', 'R3SYNTH', 'fuse.', 'Scraby', 'Sharp Man', 'Br0med', 'krassasig', 'Háski', 'Andri Már', 'Shakespeare verður ástfanginn', 'Reynir', 'EVR3stNess', 'Kolassoid', 'Akeckee', 'ZENA', 'zalagasper', 'Eliot', 'Michael Rice', 'S!sters', 'Tulia', 'Srbuk', 'Jurijus', 'Sarah McTernan', 'Flosi V', 'Screamerboys', 'Egill Stolz', 'Helgi B', 'Stefán Karl Stefánsson', 'Bergur Leó', 'Caulfield', 'ZÖE', 'Youngcuzzoo', 'Óðinn Valdimarsson', "J'adora", 'Ieva Zasimauskaitė', 'Lea Sirk', 'DoReDoS', 'SuRie', 'Flosi', '$igmund', 'Snow Dept.', 'Ragga Holm', 'iDubbbz', 'Dihaj', 'Hovig', 'IMRI', 'Jana Burčeska', 'Slavko Kalezić', 'Rúnar Eff', 'Þórdís Birna Borgarsdóttir', 'Erna Mist Pétursdóttir', 'Rakel Pálsdóttir', 'Sara Farell', 'TheoFuego', 'Frans Harju', 'Helena Routa', 'The View', 'Hussa', 'Oskr', 'DJ Massimo', 'Kuohu', 'Henriguez', 'Lxandra', 'Knipi', 'Sara Chafak', 'Raaka-Aine', 'NICHOLAS X', 'KIKI 76', 'Xtojo', 'Mauton', 'VV', 'Tomi Amor', 'Antony Parviainen', 'Toni Kakko', 'Leyton Lake', 'Frances', 'Pajafella', 'ODE', 'Mikaveli', 'Romu', 'SANÉ', 'Tre Kronor', 'Samppa Linna', 'ILLSIGHT', 'JORI SJÖROOS', 'Paul Elias', 'Malka', 'Kauriinmetsästäjät', 'Julia Rautio', 'Nelli Milan', 'Vahtera', 'Blue Street Players', 'Ronja Mariela', 'Planet SAHH', 'Kerava', 'Kid1', 'Deata', 'SHE', 'City Street Jazz Band', 'Møtions', 'MIROxNIKI', 'Epelit', 'Pasa', 'KOI', 'Rácz Gergõ', 'VALMAR', 'Kasza Tibi', 'IstenEst', 'Nks', 'Bereczki Zoltán', 'HRflow', 'Mario Judah', 'Tiah', 'Apu', 'Carson Coma', 'Strongi', 'LMEN PRALA', 'Pamkutya', 'Vanek Andor', 'Misshmusic', 'USNK', 'Senkise', 'Horváth Tamás', 'Pixa', 'Filius Dei', 'Samuel Kim', 'JustVidman', 'Red Bull Pilvaker', 'Dé:Nash', 'Konyha', 'Hundred Sins', 'King Monroe', 'AKC Misi', 'ibbigang', 'New Level Empire', 'Berkes Olivér', 'Szakács Gergő', 'Zséda', 'TNT', 'Sipos Tamás', 'Radics Gigi', 'Soulwave', 'Zävodi', 'HoneyBeast', 'LYSA', 'Opitz Barbara', 'Kollányi Zsuzsi', 'Margaret Island', '30Y', 'Trifecta', 'Kindler Shah', 'Stolen Beat', 'Molnár István', 'Egs', 'Pøla', 'Elefánt', 'Kispál és a Borz', 'Ismerős Arcok', 'Szabó Balázs Bandája', 'Magneoton Sztarok', 'Dana', 'FIZIstyle', 'Grey256', 'Skwor', 'Wesome', 'Milion Plus', 'Refew', 'HILLS 97', 'P T K', 'Adam Ďurica', 'Fobia Kid', 'Dan Barta', 'Skardu', 'ONEMANSHOW Foundation', 'Adam Mišík', 'Reginald', 'Za100let', 'Amco', 'Redzed', 'MikeJePan', 'Johny Machette', 'Janek Ledecky', 'XINDL X', 'Nightwork', 'DJ Rusty', 'Berenika Kohoutová', 'SKiNNY BARBER', 'Maniak', 'Cypis', 'Hard Rico', 'Michal Hruza', 'ATMO Music', 'Porsche Boy', 'SpecialBeatz', 'DJ Kadr', 'Dětský sbor Camerata', 'LIL GRIPPIE', 'Hugo Toxxx', 'DeeThane', 'Martin Matys', 'Naflexx', 'Forgen', 'Vadak', 'FattyPillow', 'TVTwixx', 'Dětský sbor', 'Josh Jips', 'Leoni Jut', 'Tomas Klus', 'Jakub Děkan', 'JJ Hannon', 'Sofian Medjmedj', 'JCKPT', 'Annet X', '5NAPBACK', 'STIKSTOF', 'Sara Gold', 'Sylver', 'Double Pleasure', 'Kenn Colt', 'Liefde Voor Muziek voor Rode Neuzen Dag', 'Jjakoveertien', 'The Starlings', 'DJ Nazil', 'Sean Dhondt', 'Karen Damen', 'Peter Vanlaet', 'Yung Mavu', 'Seedy', 'J Pablo', 'Twin n Twice', 'Trobi', 'Mastachi', 'Sordal', 'Amanda Wittman', '404Billy', 'bvd kult', 'Silvy', 'DJ Licious', 'Curtis Alto', 'Tavi Castro', 'Nassi', 'Natalia', 'Tim Arisu', 'D5R', 'J1', 'Tilhet', 'pajut ja muut', 'Spacc', 'the Serpent', 'spacc', 'Freddie Shuman', 'Halloran', 'Kate', 'Lil G.', 'Jickson', 'Sheen', 'PJAY CZ', 'SK'])
set6  = set(['Carl Knast', 'DIBSET', 'SunCity', 'Boris Laursen', 'Jamie Kamara', 'Børnemusik Dyrene', 'Awave', 'N3LLY', 'HAM fra Syd', 'Merro8', 'IBO', 'Berd', 'LayLay', 'Jesu Bankerz', 'Kristian Kjærlund', 'Alexander Husum', 'AMRO', 'Leprom', 'Karizman', 'Elvira Pitzner', 'VM Holdet', 'KING', 'Vkation', 'Tramper Torben', 'Place On Earth', 'Delahoia', 'Sandra Hussein', 'Soleima', 'Tore', 'Tiril', 'Henrik Blomme', 'Katie Keller', 'Salazar', 'Dj Guldsmed', 'Jytte', 'Lakserytteren', 'CHILI', 'MGP Allstarz 2017', 'Guldpige', 'JB', 'Euroo', 'Marti', 'Malaka', 'buller', 'Oskar Häggström', 'A36', 'Seedy', 'HON', 'SG Lewis', 'Style', 'Erik Lundin', 'Matte Caliste', '2.clock', 'Abidaz', 'Biggie Juke', 'Suzi P', '2M', '10an', 'Linda Olsson', 'Bailey Jehl', 'Four of Diamonds', 'Janice', 'The Mayries', 'Jelassi', 'Wild-Boy', 'Michael Rice', 'zalagasper', 'Herbert Munkhammar', 'Jonathan Johansson', 'Mila Crowell', 'Lionman', 'Leyton Lake', 'Amy Soonyoung', 'Tony Lieberman', 'Hallman', 'Max George', 'Ieva Zasimauskaitė', 'May Raya', 'Thomas Sekelius', 'Total Ape', 'Hovig', 'IMRI', 'Henrik Berggren', 'Les Gordons', 'Galavant', 'Blåsemafian', 'Jorn', 'Halva Priset', 'Ole Hartz', 'EMMY', 'Kristin Husøy', 'Magnus Bokn', 'S1sco', 'HAYES', 'Philip Emilio', 'Oskar Westerlin', 'Daskeladden', 'Kevin Boine', 'Torine', 'Roc Boyz', 'Yasmine San Miguel', 'Martin Heier', 'GoToGuy', 'GABIFUEGO', 'NALO', 'Drikkeleken', 'Simon Kleiveland', 'DMNDS', 'Matilda', 'B-Boy Myhre', 'Ida Celine', 'Kutoppen', 'Jonathan Floyd', 'Petter Northug Jr', 'Konen på torget', 'Bendik Brænne', 'Ulrike Hansen Døvigsen', 'Penthox', 'Bearson', 'Vilde og Anna', 'Dunisco', 'Future Duper', 'ARY', 'Adrees', 'Clairmont', 'Alex Ross', 'Serlina', 'Tunge Ferrari', 'Piateed', 'Olav Haust', 'Chris Haugan', 'Huemist', 'Ezzari', 'Louam', 'Alberto Ciccarini', 'ZadeKing', 'INDIIA', 'Jonas Benyoub', 'Swedum', 'Chad Cooper', 'Temmpo', 'Travelle', 'CASPR', 'Turab', 'Pojkarna', 'Marte Bratberg', 'Sara Farell', 'dePresno', 'DJ Anton', 'August Dahl', 'Atle', 'Maria Celin Strisland', 'EMIR', 'Maybon', 'Ella Lockert', 'Prebz Og Dennis', 'Stein Berge', 'Bendik', 'Lxandra', 'Tayla Parx', 'Yomi', 'Bad Geckos', 'Frank Løke', 'Hanne Mjøen', 'JRL', 'Marthe Wang', 'X Daniel', 'Øystein Hegvik', 'Meso', 'Henrix Borg', 'Kung Vladimir', 'DJ NorgeJodel', 'Mina Lund', 'Sirkel Sag', 'Juur', 'Triana Iglesias', 'INAP', 'ZL-Project', 'Mad.S', 'Ola Nordmann', 'Bernt Hulsker', 'Yung Potato', 'Thea Gudmundsen', 'Petter Pilgaard', 'Snow Boyz', 'Georgia Box', 'Britz', 'Kapteinen', 'NorthKid', 'Olympis', 'J1', 'the Dodos', 'Bea', 'Victoria', 'Dimitri Vangelis', 'Wyman', 'Halloran', 'Kate', 'Edgar', 'Meland', 'Skei', 'PT', 'Mannen', 'Zino', 'Eddy', 'Norway', 'Paradise Hotel'])
set7  = set(['Kamal Khaira', 'Madhu Priya', 'Roll Rida', 'Raghav Chaitanya', 'Simran Kaur Dhadli', 'Khushank Dalal', 'Nimrat Khaira', 'Sickflip', 'Kulbir Jhinjer', 'Nirvair Pannu', 'Bhuvan Bam', 'Korala Maan', 'Tanzeel Khan', 'Sam C.S.', 'The Landers', 'Sanjith Hegde', 'Arun Dev Yadav', 'Sarath Santosh', 'Amruta Fadnavis', 'Happy Raikoti', 'Mickey J. Meyer', 'Rupinn', 'Nitin Mukesh', 'Tamir Bar', 'Orr Amrami Brockman', 'Tay G', 'Jimbo J', 'Teddy Neguse', "100 First Songs' Participants", 'Francis Duran', 'Glen Dean', 'Føx & the Hound', 'Sergio', 'Pako Ramirez', 'Young Wavee', 'דקל וקנין', 'Berg', 'Roy Kafri', 'ZENA', 'Asaf Goren', 'Ishay Levi', 'Avior Malasa', 'Yossi Shitrit', 'Gher', 'Michael Swissa', 'אבי פאנל', 'Katrix and Doron Beaton', 'Liron Amram', 'Omri 69 Segal', 'Buttering', 'Afroto', 'Ahmed Santa', 'Mido Gad', 'Angham', 'Barry', 'Ali Adora', 'Abo Lila', 'Emad Kamal', 'Karim Mohsen', 'Rain Sleepings Sounds', 'Limit 29', 'Rain Sound Studios', 'Alex Ran', 'Barachi', 'Idahams', 'Rc $pitta', 'Majid Almohandis', 'Overlord Bnjmnz', 'T-Krazie', 'Lady Sparkle', 'Dufflebag Cal', '7098', 'Ebx Music', 'ARNOLD', 'Luigui Bleand', 'Josh Milli', 'A-RealBeats', 'Dmvc Blvck', 'Shiah Maisel', 'TYTE', 'Terri Cann', 'damnboy!', 'Luca Testa', 'Assassin', 'FEYI', 'Jorge Pensi', 'Julius', 'Yedinci Ev', 'Lvbel C5', 'Karya Çandar', 'Cem Pilevneli', 'Kodes', 'Ahiyan', 'Maho G', 'Grup M. Şanlıel', 'Irem', 'Halil İbrahim Göker', 'Gökşin Derin', '90 BPM', 'Anıl Durmuş', 'Mansa Obi', 'Timuçin Esen', 'DJ LYAN', 'Medusa', 'İntizar', 'Özlem Akgüneş', 'Sibel Alaş', 'Güler Özince', 'Betül Demir', 'Atiye', 'Pentagram', 'Nigar Muharrem', 'Deniz Gürzumar', 'Asala Maleh', 'Killval', 'Mohamed AlSalim', 'Ahmed El Maslawy', 'Aburob', 'Akihiro Manabe', 'Rap AR Anime', 'Blvxb', 'Mohmood Al Ghaiath', 'Joseph Attieh', 'Humood AlKhudher', 'Real Recognize Rio', 'Lil Eazy', 'Archez', 'WESTSIDE BOOGIE', 'Lindiwe Mkhize', 'Jafar Al Ghazal', 'Haifa Wehbe', 'Konata Small', 'the Serpent'])
set10 = set(['Sergi', 'La Mosca Tse-Tse', 'Marcos Menchaca', 'Gera MX', 'Heroes Del Silencio', 'Yuawi', 'Где Фантом?', 'Kara Kross', 'SEEMEE', 'PALC', 'ElyOtto', 'Dzharakhov', 'LIDA', 'G4OUR', 'CAPTOWN', 'Smoky Mo', 'Pacosp', 'IsaacMusic', 'Gunwest', 'Ямаджи', 'Armando Hernandez', 'Ryan Roy', 'Leebrian', 'DJ Goozo', 'LosPetitFellas', 'Dejota2021', 'Luisa Fernanda W', 'Koffee el Kafetero', 'Kenai', 'Sanluis', 'Martin Elías', 'Leo Mattioli', 'Tambó Tambó', 'Felix Gabriel', 'DJ Yayo', 'La Nueva Luna', 'Wanakin', 'Joleane', 'Joe Hasin', 'Miz Lina', 'Nadia Dan', 'Aida gean', 'Karl Michelson', 'Miles Kenn', 'Kennis Inn', 'Nickl Mons', 'Adam Dank', 'Justin Tomber', 'Edik', 'Dean Loo', 'Vitya AK', 'payton', 'Mitchel', 'Фикс', 'Gocata Kangal', 'VessoU', 'Togasang', 'Havenbled', 'Jamaicataxi7', "Leron Rankin'", 'Gcollectah', 'Crystal Byrd', 'Donika', 'Riko Band', 'LZ27', 'KNUX', 'MacShawn100', 'Nikki Luchese', 'FlameShadow', 'SayReal', 'NEEL', 'Leol', 'CashnOut Esco', 'StzCorp', 'VICTORIA', 'Ran Nishihara', 'Jamiryo', 'Josh Jips', 'Leoni Jut', 'Kindler Shah', 'Venata', 'Iskrata', 'Luca Testa', 'JJ Hannon', 'TIFFANY BYNOE', 'Hones-T', '13 Crowns', 'Exiled', 'iDubbbz', 'Kyle Edwards;Chris Bevrly', 'Kyle Edwards;DJ Smallz 732', 'Angerwolf', 'Miquela', 'Cassow', 'Ian Erix', 'Chris Tyson', 'Kid3rd', 'Dylan Diamond', 'Cal Scruby', 'Plamen', 'Ricegum', 'JRY', 'Fran Rozzano', 'Dowba Montana', 'Yiyo Zarante', 'Deiby Lceja', "Leyenda Barrial Boy'z", 'Jay Menez', 'Alex R.', 'El Chuape', 'Sergio Echenique', 'J Quiles', 'Lr Ley Del Rap', 'La Manta', 'Jonzmen', 'Hey Santana', 'Nico Clinico', 'Danay Suárez', 'El Batallon', "D'Markuz", 'Chystemc', 'La Sonora De Tommy Rey', 'Grupo Alegria', 'Agrupación Marilyn', 'Ak 4:20', 'Flor De Rap', 'joel music', 'Músicxs De Chile', 'Ankhal', 'Bayronfire', 'Conjunto Alerce', 'Daniel Munoz', 'Hector Pavez', 'Los de Ramon', 'Jorge Yanez', 'Julianno Sosa', 'Eyci and Cody', 'La Combo Tortuga', 'Mym', 'Leka el Poeta', 'Jona Camacho', 'Roberto Parra', 'Los Hidalgos', 'Hermanos Morales', 'Yulien Oviedo', 'Fiesta Huasa', 'Ballet Folklórico Alma Chilena', 'Power Peralta', 'Ronald El Killa', 'Marvell Boi', 'Romeo El Santo', 'Marvel B', 'Portavoz', 'Tracy McGrady', 'Paithon', 'Atomic Rose', 'Kazzabe', 'Steven Pineda', 'J Calix', 'Nassir Chahin', 'Nio Nassar', 'JWolfy', 'Juan Oliva', 'Angel Gutiérrez', 'Rodolfo Bueso', 'Jesus Aguilar', 'Macario Mejia', 'Los Bohemios', "What's Up", 'andrei', 'Sapte', 'Mario Fresh', 'Alberto Grasu', 'Iobagg', 'Oscar', 'Roxen', 'VUNK', 'Mario Judah', 'RAVA', 'Florian Rus', 'Super Ed', 'Lino Golden', '5GANG', 'Yakki', 'Dorian Popa', 'SICKOTOY', 'ADDA', 'Stefan Hrusca', 'Deliric', 'Jay Watts', 'Lola Brooke', 'Drew Allen', 'Corey Mekell', 'Mark Stam', 'Aerozen', 'boi the creator', 'Inyeewetrust', 'Son Orioz', 'Mr. Glasses', 'Subcarpaţi', 'Speak', 'Calinacho', 'NMW', 'El Nino', 'Ali Dolla$', 'TRISHES', 'Olivia Sabates', 'Doddy', 'Gloria Prince', 'P.M.', "Beau'Joli", 'Lovel', 'BAZOOKA', 'Chris Bones', 'DOC', 'Real Imaj', 'Knack', 'NxtSkool', 'Omar Courtz', 'Pitizion', 'Bebo Yau', 'Tres Dedos', 'Arevalo', 'Durako', 'Bombotunes', 'La Máquina Camaleón', 'Dennis Fernando', 'Navigator', 'Eshconinco', 'DeStorm', 'Dj Coss', 'Dancehall Word', 'Blaiz Fayah', 'DJ Kendo', 'GIMARIO', 'Alex Badilla', 'Pranz', 'Fátima Pinto', 'MAYKEL', 'SpecialBeatz', 'Porsche Boy', 'Frayer Flexking', 'Momo', 'Grey256', 'Fobia Kid', 'AstralKid22', 'Martin Agh', 'Duch', 'Pavol Habera', 'Lubomír Lipský', 'Dominika Mirgova', 'Janek Ledecky', 'Marie Rottrova', 'Robo Opatovsky', 'Luca Brassi10x', 'Cypis', 'ONEMANSHOW Foundation', 'Čis T', 'Adam Ďurica', 'LUISA', 'Radikal Chef', 'Šorty', 'KHANS', 'Milion Plus', 'Kruel', 'Redzed', 'Tina', 'Loko Loko', 'Falusko', 'Miko Floso', 'Taomi', 'Alan Murin', 'Nicholas Naison', 'Samuel Kim', 'Rida Radar', 'Mairee', 'Marián Kochanský', 'Simon Smith', 'Peter Simon', 'Naflexx', 'TKX', 'Slimso', 'Annet X', 'Ultrazvuk', 'For You Acapella', 'P.A.T.', 'DJ Rusty', 'Supa', 'Trifecta', 'Lukas Adamec', 'LYSA', 'Peter Pann', 'Deyvis Orosco', 'Cielo Torres', 'Maricarmen Marin', 'Armonia 10', 'Claveles de la Cumbia', 'Faraón Love Shady', 'Combinacion De La Habana', 'Temple Sour', 'Orquesta Caña Brava', 'Orquesta Bembé', 'Nicole Favre', 'Anna Carina', 'Susan Ochoa', 'Reggaeton bachata Hit', 'Dani Darcourt', 'VICE MENTA', 'Hnos Yaipen', 'DJ Krlos Berrospi', 'Orquesta Caribeños de Guadalupe', 'Marco Romero', 'DJ Luigi', "Pelo D'Ambrosio", 'Cruz', 'Mario Hart', 'Av. Larco', 'Orquesta Zaperoko del Callao', 'Korina Rivadeneira', 'BCA', 'Dubosky', 'El Codigo Kirkao', 'Robinho', 'Towers Mix', 'Señor Loop', 'Bobo', "At' Fat", 'Yemil', 'Kabliz', 'El Kid', 'Sofía Valdés', 'Chamaco', 'El Zeta', 'MOI MUSIC', 'Mista Bombo', 'Fulo El Yeyo', 'T.O.T', 'Robi Guid', 'El Boza', 'BARBEL', 'Jhon el Diver', 'Amindi', 'Los Topsys', 'Original Fat', 'Azhika', 'Rawayana', 'Real Phantom', 'Saulės Kliošas', '7th Block', 'Šventinis bankuchenas', 'Jessica Shy', 'Furytto', 'Keanu Blunt', 'Angelou', '2Kvėpavimas', 'Velvet', 'GJan', '335d', 'Flying Saucer Gang', 'Zoliukas', 'Solo Ansamblis', 'ilyTOMMY', 'Gebrasy', 'wavytunesz', 'Gabrielius Vagelis', 'Daddy Was a Milkman', 'Evgenya Redko', 'Eyedress', 'Beatrich', "Radistai Dj's", 'Monique', 'QV Trap', 'Shouse', 'Jovani', 'GAZIROVKA', 'Ieva Zasimauskaitė', '8 Kambarys', 'Alekna', 'Saulius Prūsaitis', 'Flash Voyage', 'SADBOI', 'Free Finga', 'Vidas Bareikis', 'Vaidas Baumila', 'Mark Les', 'senjasa', 'AREZRA', 'Kamaniu Silelis', 'jautì', 'Swertas', 'OSVISS', 'Abudu', 'NKSN', 'Kriminell', 'Gabrielė Vilkickytė', 'Monika LIU', 'MITCH RECORDS', 'Tinlicker', 'Monika Linkyte', 'On1Beat', 'Junior A', 'ba.', 'Мэвл', 'Audioiko', 'Rūta Loop', 'Monika Marija', 'MEANDI', '3-ий Январь', 'Radvilas RA', 'MR.Nuclear', 'Golden Parazyth', 'Iglė', 'zalagasper', 'Jurijus', 'Karolina Meschino', 'Carousel', 'ZENA', 'Eliot', 'S!sters', 'Srbuk', 'Michael Rice', 'Sarah McTernan', 'Ester Peony', 'Tulia', 'Garbanotas', 'Dog Collective', 'Lea Sirk', 'SuRie', 'Ieva ZasimauskaitÄ—', 'Laura Rizzotto', 'Eugent Bushpepa', 'DoReDoS', 'Hiperbole', 'Baltasis Kiras', 'Dihaj', 'IMRI', 'Hovig', 'Triana Park', 'Jana Burčeska', 'Timebelle', 'Artsvik', 'Kasia Moś', 'NAVIBAND', 'Tijana Bogicevic', 'Slavko Kalezić', 'Norma John', 'The View', 'Tim Legend', 'Garotihnio', 'Mariano Bermudez', 'Cuatro Pesos de Propina', 'Sonido Caracol', 'Karibe con K', 'Once Tiros', 'Chacho Ramos', 'Los Pikantes', 'Los Negroni', 'Buitres', 'Jotape', 'La Sandonga', 'Valeria Gau', 'Marcos da costa', 'Gian', 'Zeballos', 'Américo Young', 'Zanto', 'Blonda', 'CB Records Music', 'Arrancandonga', 'Miway', 'Edu Lombardo', 'Canario Luna', 'Mart', 'Sonido Cristal', 'Lafame', 'Sonido De La Costa', 'Urband', 'The Panas', 'Dessia el Otro', 'Halconizados de Amor', 'Corta La Bocha', 'Estrategas', 'DeGira', 'Tu Papa', 'Enero', 'RC BAND', 'Mala Tuya', 'La Mano Rancia', 'Los Verduleros', 'Cumbia Juan', 'Chili Fernandez', 'Japiaguar', 'Talento de Barrio', 'Noa - Noa', 'Escucha!', 'Samuray', 'Tropical Caribe De Los Hnos Loera', 'Los inquietos del vallenato', 'Pipa Para Tabaco', 'Tropical Caribe', 'La De Roberto', 'LaNuestra', 'Lenuel', 'Bryant Mayers', 'Joshua Dietrich', 'MC LEO', 'Lucas Cliff', 'Tierra Adentro', 'Afaz Natural', 'BB Nobre', 'Reggaetones', 'Villagrán Bolaños', 'Em Rossi', 'Drew Cole', 'Dj Maya', 'Manifiesto Urbano', 'Carin', 'StephenFlint24', 'ScottHall41', 'JulieHarder67', 'CarolAnderson88', 'MariaBrewer62', 'CecilDowns84', 'IsisWampler68', 'Temry', 'CynthiaGorecki58', 'S I', 'Sean Matsukawa', 'Midnight Office', 'D.K. Smith', 'Destripando la Historia', 'Akil Hamilton', 'Michele 69', 'Alan 1', 'Alan 2', 'Tuan Alto Sax', 'Doris Schmeler', 'Dorothea Berger', 'Adonis 88', 'TINA LTT', 'Adonis 66', 'Lucy Beahan', 'Veronica Effertz', 'Joseph Murray', 'SarahStiltner', 'Eula Glover', 'HenryNorrell', 'JamesPonder', 'TerryFerrell', 'Slender2003', 'The Face 1234', 'Indifferent2527', 'Lucky2135', 'Coldly1989', 'Vuvanhy1985', 'Wistful2010', 'Happy9282', 'Silence2001', 'Fragile1988', 'Comfortable5343', 'TonyJones', 'Magnolia1982', 'Sunset2002', 'Marty McKay', 'CaryLamb', 'RonTurner', 'JeffreyLewis', 'Weldon S. Kesterson', 'EdithFernandez', 'DianeKennedy', 'Michele 99', 'Doris 68', 'Harold 69', 'Harold 68', 'Doris 666', 'Haring 68', 'Dong Hung', 'Aria Ohlsson', 'DorisAlvarez', 'JillReece', 'PaulaSanchez', 'ChadCaver', 'AndreaRuby', 'VeraQuintana', 'GeorgeGoin', 'DanaHuntley', 'BennieNolen', 'LindaCourtney', 'MarySchroeter', 'SusanMiller', 'JanieReaves', 'SusanCarter', 'RickSpencer', 'PennyNakamura', 'LouChamberlain', 'micem', 'lil waii', 'NorsePrez', 'Chika Toro', 'YT', 'Anne Dorko', 'Vashoun Stjon', 'Talley', 'Catswithnicefaces', 'King V', 'Streetza Ria', '$hea Butta', 'Phillydark1', 'Xciryl', 'Legend Russell', 'Trilltypefella', 'Philip Anthony', 'Rockin Chefs', 'Tim Hustle', 'Tiny Tree', 'Tom. G', 'Michael Sterling', 'Flood District', 'Saviour3.0', 'OGK John', 'JOSEPH CHASE', 'His Kings', 'ThereGoJo', 'The ocn', 'SIXN', 'D14', 'Moonlight Young', 'Mizzery', 'Jeremy Voltz', 'Keith Lawsen', 'Thomas Deil', 'Samylon92', 'Melanie Weichl', 'Splash Danero', 'Decayer', 'Inas X', 'Mahkenna', 'Eastside Jody', 'Tripple 2', 'Raja Rose', 'Dragana Kosic', 'Christos Mavridis', 'Duo Guardabarranco', 'Zoragoza', 'Top Mixer Dj', 'Perrozompopo', 'Julio Melgar', 'Tayl G', 'Viento En Contra', 'Aliento', 'Los Abejorros', 'Danny Marin', 'be vis', 'Maciej Zielinski', 'Olszakumpel', 'Julia Rocka', 'Josef Bratan', 'Kamil Bednarek', '2K', 'Ania Dabrowska', 'Piotr Ziola', 'Popkiller Młode Wilki', 'Męskie Granie Orkiestra', 'Mela Koteluk', 'Daj To Głośniej', 'Muniek Staszczyk', 'DMN', 'Mlodyskiny', 'Syberia Tdw', 'Eis', 'Lanberry', 'Szymi Szyms', 'Kacper HTA', 'Jacuś', 'Szczyl', 'Julia Wieniawa', 'TUZZA Globale', 'Donguralesko', 'Fresh N Dope', 'WIP Bros', 'QueQuality', 'Marcin Maciejczak', 'Kacper Blonsky', 'Jetlagz', 'Stanisław Soyka', 'Edyta Gorniak', 'Katarzyna Cerekwicka', 'TGD', 'Roksana Węgiel', 'Young Slash', 'Lipa', 'Kamerzysta', 'Jacek Stachursky', 'Fisz Emade Tworzywo', 'Grzegorz Hyzy', 'Magda Steczkowska', 'Mate.O', 'Roan', 'Pamela Stone', 'Agnieszka Chylinska', 'Dj.Frodo', 'Podulka', 'Xxanaxx', 'Męskie Granie Orkiestra 2017', 'SexMasterka', 'LASERY', 'G-Girls', 'Aniceto Molina', 'Los Hermanos Flores', 'Orquesta Internacional Hermanos Flores', 'Comando Tiburon', 'Cumbias Viejitas', 'La Divaza', 'Planeet', 'Andrei Zevakin', 'Bru-C', 'Jüri Pootsmann', 'Grete Paia', 'Maian', 'NOËP', 'Heleza', 'Genka', 'marp$', 'KiROT', 'Raul Ojamaa', 'Jaagup Tuisk', 'kaw', 'Siimi', '372kaspar', 'WATEVA', 'Terminaator', 'Beatrice', 'Maxtract', 'Cityflash', 'Lemon Fight', 'Arop', 'Vajé', 'Leslie Da Bass', 'Öed', 'Sip€lga 14', 'Doc Ketamer', 'okeiokei', 'DJ Koze J', 'Red Tan', 'Paul Ereck', 'Emy Blu', 'Lumevärv', 'KOIT TOOME', 'silverrari', 'fenkii', 'Merilin Mälk', 'Merlyn Uusküla', 'Jozels', 'säm', 'Eleryn Tiit', 'Mr. Happyman', 'Ines', 'Uku Suviste', 'Cdeep', 'Frankie Animal', 'Egert Milder', 'Anett', 'STEFAN', 'INGER', 'Uudo Sepp', 'PSL', 'I AM THE STORM', 'Signs', 'Göki', 'Asena', 'Kerli Kivilaan', 'Xtra Basic', 'Kadiah', 'Sissi', 'Lacy Jay', 'Johanna Eendra', 'Daniel Levi', 'Indrek Ventmann', 'Metsakutsu', 'Miljardid', 'Karl-Kristjan', 'Iiris', 'Hoax', 'Whogaux', 'Angeelia', 'AZMA', 'Cruz Santa', 'CXB', 'LindaTirado', 'SummerFoster', 'Jason William', 'LauGadoi', 'EmilyZimmer', 'Grandha', 'Juan Solo', 'Mayte Flores', 'Luis Vega', 'Diego Merritt', '$ourJ', 'Singapūras Satīns', 'ansis', 'Zigfrīds Muktupāvels', 'Grafomaans', 'Intars Busulis', 'Prusax', 'rolands če', 'Bermudu Divstūris', 'OLAS', 'GACHO', 'Mesa', 'New Wavy', 'Sudden Lights', 'Aija Andrejeva', 'VIŅA', 'Kreisais Krasts', 'Adler Kotsba', 'Livi', 'Putnu Balle', 'Ingus Ulmanis', 'Bet Bet', 'Musiqq', 'Lauris Reiniks', 'Zvaigžņu koris', 'Projekts “Tev tuvumā”', 'Eināra Verro Vokālā Grupa', 'Otra Puse', 'Tev tuvumā', 'Fomins', 'Arturs Skrastins', 'Kiol', 'MBR', 'Emil', 'Forrest Hill', 'morgxn', 'The Key Rocks', 'Kenedi', 'Post Ironic State', 'Steven Rosvell', 'Dior', 'Perkons', 'Apvedceļš', 'Menuets', 'IGO', 'Jātnieki', 'Bruģis', 'Latgalīšu Reps', 'Ferrum', 'Hospitāļu iela', 'Jumis', "Astro'n'out", 'Autobuss debesis', 'Dālderi', 'Metronoms', 'Reinis Sējāns', 'Ainars Mielavs', 'Mart Inc.', 'Kastrācija', 'Sharlot', 'Samanta Tina', 'Trench Mafia Locco', 'Āris un Saulceris Ziemeļi', 'Luda', 'Lil Karlix', 'Edavārdi', 'Simon Riemann', 'London Richards', 'Matt B', 'Kaio Kane', '7eit', 'Fakts', 'MADARA', 'Markus Riva', 'Symon', 'Young DrA', 'Monsieur Job', 'MaWayy', 'Gotti Mirano', 'Nick Hissom', 'Los Profetas', 'Trashock', '24Heavy', 'Jvnior', 'And Then Came Fall', 'Ladotee', 'OFMB', 'Big Sim', 'Brian Ligon', 'Tinny Mafia', 'SAINTS', 'Danny Blue and the Old Socks', 'Hypothetical', 'BowlLane Slick', 'Najuah', 'DJ Smallz 732', 'Mariño', 'Lexz Pryde', 'Rod Maine', 'Tony Chevy', 'Ashley Lovechild', 'Stix', 'The Ludvig', 'Santa Danelevica', 'Miks Dukurs', 'Toms Kalderauskis', 'L.V. Lauris Valters', 'Crime Sea', 'Miks Galvanovskis', 'Ziemassvētku zvaigžņu koris', 'Su Orquesta', 'Su Apollo Sound', 'R.K.M', 'los Fundamentalistas del Aire Acondicionado', 'Waor', 'Natos', 'SAMMY', 'LESEN', 'Ivo', 'Peeva', 'Yomel', 'Silvia Infantas', 'Suro Solar', 'Camila Recabarren', 'Dash', 'Cangri', 'Urba', 'Don Medardo', 'Sus Players', 'Jickson', 'Sheen', 'Halloran', 'Kate', 'Josimar', 'Su Yambú', 'JOSIMAR', 'SU YAMBU', 'Samy', 'Beissoul', 'Einius', 'G', 'G Sindikatas', 'El Gucci', 'Su Banda', 'Tom', 'Collins', 'El High', 'Dayme', 'Néstor Ló', 'Feer', 'Lees', 'LEILAH SAFKA', 'FREDDY FENDER', 'Rafał Borzozowski', 'Małgorzata Burdzel', 'Sound`N`Grace', 'RL9', 'La Luz Roja De San Marcos', 'Mäx', 'Clicherik', 'Küberünnak', 'Karmo', 'The Sun', 'Andris Erglis', 'I MEAN LOVE', 'chris nov', 'CLYDE', 'Cherry'])
set11 = set(['04 Limited Sazabys', 'SPiCYSOL', 'hachi', 'ACE COLLECTION', '稲村オーケストラ', 'PRODUCE 101 JAPAN', 'ヒプノシスマイク -A.R.B- (Division All Stars)', 'Whiteberry', 'CHIAKI SATO', 'AYA A.K.A PANDA', 'THE ORAL CIGARETTES', '浜田雅功と槇原敬之', 'Hypnosis Mic -D.R.B.- (MAD TRIGGER CREW)', 'Kinokoteikoku', "The Urashima's", 'Fumiya Fujii', 'Mone Kamishiraishi', 'Takehara Pistol', 'Ovall', 'Straightener', 'DOBERMAN INFINITY', '吳林峰', 'Panther Chan', 'Cath Wong', 'Yan Ting', 'Edan 呂爵安', 'my little airport', '石山街', 'Adason Lo', 'Thomas DGX YHL', '癲佬 dlo', 'Akano', 'YNW Jordan', 'Floating in Space - Project', 'Big Loso', 'Windy Guai', 'DNA', 'PBE PLUTO', 'Jt el Utility', 'Damián V', 'Sirvan Khosravi', '井谷俊二', 'MariKuroso', 'Mukaybin Mukayze', 'King Gunna', 'Zoee8', 'Bogart Bonales', 'Dennis Fernando', 'Friday Santana', 'Harj Savage', 'bruno livesax', 'JSnake', 'Keyzs-Sean Sparsfa', 'From Under Concrete Kings', 'ED LIIT', 'FAO>dino', 'Buunkin', 'Los Necesarios', 'TG', 'Phantom Caine', 'Molina Molina', 'Thee JAE', 'Kiian', 'DeAnna Stealth', 'Luca Natale', 'Marli Space', 'Torito', 'K Beezy', 'Rachel Mason', 'Abrina', 'LIL BLU£PRINT', 'Kamar Tachio', 'Juke', 'Bradley Mantha', 'Nuski Baby', 'Idahams', 'Duane Flames', 'J.Day', 'Grupo Okixem', 'Srene', 'RealDeal Berns', '3NIGMA', 'King Thateron', 'Said P.', 'ONSC', 'Levi Belle', 'Orphée La Fine', "Niki's Project", 'The Greywolf', 'Vaolight', 'MM', 'Lipford', 'Anzo Weiseil', "YB D'nero", 'Geek-TheHot', 'Cade Eliason', 'Acezz', 'Dario Puleo', 'Musas', 'Frank Briones', 'Moniris', 'Jalayan', 'Ghost Chili', 'JC the Don', 'Mark$man', 'Dsean', 'ViibesFromNetto', 'Fantasy', 'Hanujay', 'Poko', 'Nick Nittoli', 'DON Kilam', 'Michael Hodges', 'Maxim Schunk', 'T. McCrae', 'Preston Hudman', 'MASON', 'Hamah Music', 'zelinen', 'Big K. The Komik', 'Roma Vander', 'Aquarius Jones', 'Lil Trizzyverse', 'Profound Artist', 'Preston Hudman TG', 'SV Stallone', 'Sacal', 'Taylor Cash', 'Lexz Pryde', 'Malik Vadheim', 'Denver Grey', 'Sparksi', 'Hirotoshi Furuya', 'JR', 'Mike Plain', 'Mawat', 'Big Wade', 'Richi Drag', 'John Daniels Band', 'Shannon K', 'BiskBoy', 'Kram', 'Darren Michael Boyd', 'Juandisemo', 'Blohk$r33fa', 'The Notorious Rio', 'No More Vips Allowed', 'Mote Dogg', 'Lil Lee', 'Def Rock', 'Atômico Mc', 'Yady', 'Hk6od6ratt 44', 'Icy Sabbath', 'COSMO.', "Lai'jhon Hawkins", '徐天佑', 'Axel Thesleff', '曾樂彤', 'Datmaniac', 'Oda Kirkwood', 'Mayar Jenisk', 'Missing Seoul', 'WAVYVIBE', 'Chan Se Park', 'Hoang Muoi', 'Quang Long', 'Mr.Siro', 'NHD media', 'Khoi My', 'Hong Nhung', 'Chi Dan', 'Wanbi Tuan Anh', 'Phuong Dung', 'Fay', 'JSOL', 'DatKaa', 'T.R.I', 'Khói', 'Xuân Nghi', 'Phạm Toàn Thắng', 'JINU', 'Thinh Suy', 'Thanh Hung', 'S.T Sơn Thạch', 'Ai Phuong', 'DEAN', 'Linh Cáo', 'JayKii', 'Phạm Anh Duy', 'itsnk', 'Hoà Minzy', 'Hoang Minh Tuan', 'VA', 'Huu Khuong', 'Nhu Quynh', 'Luu Anh Loan', '菲道尔', 'GJ', '卓義峯', '曾博恩', 'Fang Wu', '唐禹哲', '劉以豪', 'TAEMIN', 'Trouze', '颜人中', 'Hua Chen Yu', 'ABAO', '莫宰羊', 'Sharon Kwan', 'Cindy Yen', 'PiA', 'Ruth Kueo', 'KUN', '采子', 'Zooey Wonder', '?te', '鄭興', '鍾明軒', '李佳歡', 'Enno Cheng', 'Barry Chen', '王貳浪', '老莫 ILL MO', 'SEJEONG', '謝博安', 'Trash', '王俊凱', 'Chyi Chin', 'Zhang Zhen Yue', '康樹龍', 'Evan Yo', '路嘉欣', 'AJ Mitchell', '陳雪燃', 'Ella Chen', 'TroutFresh', 'Queen Wei', '邵雨薇', 'Da Zhuang', '黃奕儒Ezu', '187INC', 'Fran', 'Meeia Foo', '安溥 anpu', '呂薔Amuyi', 'Highlight', '陶晶瑩', 'Christine Fan', 'Dongye Song', 'HUSH', '柯智棠', '陳珮茹', 'C.T.O', '楊培安', '肖懿航', 'Anni Hung', 'Crispy脆樂團', 'Sam Lee', 'MXM', '謝金燕', '林生祥', '馬雲', 'Yuzi Qin', 'YOONA', 'Suming', '陳惠婷', 'Deserts Chang', 'Soo Wincci', '小S', 'IT’S', 'Bolly', '老鬼團', 'Xian Zi', 'Twinko', '鄭興琦', 'Rain Lee', '簡弘亦', 'Tien Fu Chen', '嚴藝丹', '殷悅', 'SEOHYUN', 'Danupol Kaewkarn', 'THE RUBE', 'พีธ พีระ', 'DarkFace', 'ลำเพลิน วงศกร', 'VKL', 'Boy Sompob', "Yes'sir Days", 'ตุลานันท์ นรเศรษฐ์พิศาล', 'อ๊อฟ ชัยนนท์', 'ONEONE', 'Gx2', 'Valentina Ploy', 'Jay Phitiwat', 'ALREADY DEADD', 'ปรีชา ปัดภัย', 'Sweet Mullet', 'Crayon', 'Daily Vitamin', 'Napat Injaiuea', 'เด็กเลี้ยงควาย', 'TOSSAKAN', 'Jus2', 'MuchMellow', 'Gyeon', 'GT', 'Catherine', 'Kindler Shah', 'gugudan', 'เทิดศักดิ์ จันทร์ปาน', 'Aeh Syndrome', 'SummerWish', 'Memory Fabricators', 'น้อย พรู', 'T-BIGGEST', 'Yang Da Il', 'นนน กรภัทร์', 'Ruj Suparuj', 'Chaleeda', 'ไอซ์ ปรีชญา', 'The 38 Years Ago', 'PETITE', 'Tanatat Chaiyaat', 'Sukrit Wisetkaew', 'Alfred Sim', 'Robert Philip', '劉瑞琦', 'ATEEZ', '魏奇奇', 'Ashland Are', 'Mega Guar', 'Festi Baal', 'ABANGSAPAU', 'Benjamin Kheng', '曲肖冰', 'TheSmartLocal', 'Ayo97', 'Yung Raja', 'RRILEY', 'BIGDBANGLA', 'Fariz Jabba', 'MYRNE', 'brb.', 'Ffion', 'Rave Republic', 'Katherine Ho', 'Sezairi', 'Venessa Fernandez', 'Haneri', '小潘潘', 'Luqman Podolski', 'Neeta', 'CHA NI (SF9)', 'Astro 全体艺人', 'Harith Zazman', 'Najwa Latif', 'Irfan Haris', 'Tajul', "As'ad Motawh", 'Atmosfera', 'Putera Band', 'Sufi Rashid', 'Zizan Razak', 'Hez Hazmi', 'Dadilia Band', 'Aizat Amdan', 'NYK', 'Hyper Act', 'XPOSE', 'Achey', 'Yazid Izaham', 'Ayda Jebat', 'Aman Aziz', 'Ben Ladin', 'Amylea', 'Wani', 'Benzooloo', 'TreeHill', 'Sheryl Shazwanie', 'Isma Sane', 'Safiey Illias', 'Fieya Julia', 'Azarra Band', 'Eza Edmond', 'Adira Suhaimi', 'ALYPH', 'Talitha.', 'DOLLA', 'Ippo Hafiz', 'Shila Amzah', 'Datuk Sharifah Aini', 'Puan Sri Saloma', "Dato' Sudirman", 'Datuk Ahmad Jais', 'Noorkumalasari', 'Junainah', 'Tan Sri P. Ramlee', 'Aman Shah', 'Nyanyian Ramai', 'Azlina Aziz', 'Fazidah Joned', 'Sanisah Huri', 'Cenderawasih', 'Rosemaria', 'Aishah', 'Ning Baizura', 'GARD WUZGUT', 'The New Boyz', 'De Fam', 'Harry', 'Drama Band', 'STARLYTE', 'Dinda Permata', 'Kmy Kmo', 'Han Byul', 'Midnight Fusic', 'Kuizz', 'lost spaces', 'DAPHNE D調', 'Warner Music Malaysia All Star', 'Alextbh', 'Spuds', 'Julia Duclos', '邱诗凌', 'Fazura', 'Masdo', 'Santesh', 'Cherpen Band', 'Fara Hezel', 'Shaa', 'Fattah Amin', 'Khairil Johari Johar', 'Gamma1', 'Ezad Lazim', 'Harris Baba', 'Laudya Cynthia Bella', 'Black', 'Sakura Band', 'yui FLOWER FLOWER', 'ミゾベリョウ odol', 't', 'H Jungle', 'STEADY', 'CO.', 'The GoGo Tuner Family', 'Mike Mostert', 'Creme', 'Caf', 'E', 'SUPER JUNIOR-D', 'Ellina', 'The Majistret', 'Megat'])
set12 = set(['DJ Stokie', 'Bok Van Blerk', 'Dj Obza', 'Blaqnick', 'Emtee', 'DJ Jaivane', 'Vhudie', 'Mthunzi', 'Loufi', 'ANATII', 'K.O', 'David Scott', 'ILuvJupiter', 'Busiswa', 'GeeSixFive', 'Costa Titch', "Soweto's Finest", 'Yanga Chief', 'Shane Eagle', 'Nija', 'Stillaround', 'Salatiel', 'Gemini Major', 'Midnight Meetings', 'ABS-CBN Music All Star', 'Jana Garcia', 'Brian Pepito', 'Geo Ong', 'Reese Lansangan', 'Amber Leigh Irish', 'Andrea Babierra', 'Kiana V', 'Hatemost', 'ItsOnlySkillz', 'TiÃ«sto', 'Ofili', 'Popayedi', 'Vercetti', 'Natassa Bofiliou', 'Ripen', 'Ethismos', 'Alcatrash', 'Konnie Metaxa', 'Young Tulio$', '12os Pithikos', 'Rio', 'Athliens', 'Kanon', 'Arva', 'Pournaras', 'Eaze', 'Natasha Kay', 'Smuggler', 'Joker Two-Face', 'ZENA', 'Eisvoleas', 'Efta', 'Panos Mouzourakis', 'Josh Jips', 'Leoni Jut', 'Rube Claws', 'Kindler Shah', 'Trifecta', 'JJ Hannon', 'Vangelis Kakouriotis', 'LYSA', 'Hovig', 'Dihaj', 'Madstreet', 'Khifnu', 'CHA NI (SF9)', 'Dhika Fawaz', 'YUJU', 'Keisya Levronka', 'Cannabis Club ATL', 'Karina Christy', 'Adrian Khalif', 'Maria Simorangkir', 'Aldo Longa', 'Ustad Jefri Al Buchori', 'Cassandra', 'Syakir Daulay', 'Acha Septriasa', 'Soundwave', 'Arnon', 'Ari Lasso dan Bunga Citra Lestari', 'Cuco', 'Crayon', 'YoWis Ben', 'Ghea Indrawari', 'Sandrina', 'Ramengvrl', 'Armand Maulana', 'Gisel', 'Dash Uciha', 'Aliando Syarief', 'Adam Mitchell', 'Kemal Palevi', 'Salshabilla', 'Inkonnu', 'Snor', 'Tagne', 'L7or', 'ElGrandeToto', 'Abdeelgha4', 'Smallx', 'Foufa Torino', 'Anys', 'Lsan L7or', 'kouz1', 'Tawsen', 'Cravata', 'ISSAM', 'Dada', 'Mok Saib', 'Djalil Palermo', 'Draganov', 'Mocci', 'Lbenj', 'Didine Canon 16', 'LFERDA', 'Hatim Ammor', 'Tiiwtiiw', 'Mehdi Mozayine', 'Madd', '7-Toun', 'Didou Parisien', 'OUENZA', 'Fat Mizzo', 'Dollypran', '21 Tach', 'Furelise', "L'morphine", 'Salma Rachid', 'Adil Assil', 'Moro', 'Mrabet', 'BrownY Original', 'Cheb Wahid', 'Loun', 'La Releve Maghreb', 'Lembawe9', 'Dioscures', 'SecondT3', 'Ayoub Africano', 'Anas', '7liwa', 'Cheb', 'Naar', 'DYSTINCT', 'Zakaria Ghafouli', 'Cheb Bello', 'Cheba Souad', 'Sanfara', 'Nouamane Belaiachi', 'Ihab Amir', 'Shayfeen', 'Qaayel', 'Cyborg', 'DJ Adel', 'MiniByte', 'Bouchra', 'Cheb Houssem', 'Ily', 'Jaylann', 'Aminux', 'Fnaire', 'Hind Ziadi', 'Aezaddy', 'Flenn', 'Shobee', 'A.L.A', 'Faycel Sghir', 'Cheb Akil', 'Liamsi', 'Drizzy', 'Ayoub Hamdaoui', 'Profit Za3im', 'Raste', 'Khtek', 'Yanso', 'Hablo', 'Mohamed Adly', 'Asmae', 'Skaymen', 'Tflow', 'Ali Ssamid', 'JenJoon', 'Barry', 'Zaynab', 'Nizoo', 'Cheb Djalil', 'Flipperachi', 'Master Sina', 'Cheb Bachir', 'Klay BBJ', 'Si Lemhaf', 'Artmasta', 'Cheba Warda', 'Manel Hadli', 'Hamouda', 'Dounia', 'H-Kayne', 'Faycal Mignon', 'Mido Belahbib', 'Psychoqueen', 'Youssef Aoutoul', 'Narco Polo', '7-toun', 'Bilal Sghir', 'Mo Temsamani', 'Anas Guzman', 'Imad Benaomar', 'Bad Flow', 'Ibtissam Tiskat', 'Adel Sweezy', 'Mr. Crazy', 'The S', 'Luigui Bleand', 'Chemsou Freeklane', 'Nizar Idil', 'Barachi', 'Spleux', 'Bilal Africano', 'Limit 29', 'Safae Hbirkou', 'Fayçal Mignon', 'Tortoz', 'Echbiy', 'Ghazir', 'Redone Berhil', 'D14', 'micem', 'lil waii', 'Kobb', 'Mathu', 'Hichem Smati', 'Savage Plug', 'Cheb Abbes', "Moov'art", 'Halloran', 'Kate'])
set13 = set(['Biu do Piseiro', 'Nêgo Jhá', 'OIK', 'Camila Loures', 'DJ Pernambuco', 'Anne', 'Rafael Allmark', 'Maycon e Vinicius', 'Thiago Carvalho', 'Fernanda Salgado', 'Thascya', 'Gabriel Elias', 'Furacao Love', 'Seakret', 'Junior Lord', 'Yuri NR5', 'Greg Ferreira', 'Fumaxa', 'MOBBERS', 'Gama WNTD', 'Yung SpaceGhost', 'WUANT', 'xtinto', 'SYRO', 'Instinto 26', 'Kappa Jotta', 'WiNDOH', 'Aragão', 'Mizzy Miles', 'Gson', 'Timor YSF', 'Achero', "Dino d'Santiago", 'Lhast', 'LON3R JOHNY', 'Irina Barros', 'Minguito 283', 'Dream Boyz', 'Betsy David', 'Waze', 'Valas', 'Domi', 'Mastiksoul', 'João Sousa', 'Miguel Alves', 'Apollo G', 'ÁTOA', 'sora9k', 'Joint One', 'Vado Más Ki Ás', 'Chris Tamayo', 'ARTIGAZZ', 'Jvst Fly', 'GROGNation', 'Mota JR', 'YUZI', 'prettieboy johnson', 'Huzz', 'Ray Polanco', 'Sea', 'Kazzio', 'Ella Nor', 'ANDREZO', 'Mundo Segundo', 'Zara G', 'April Ivy', 'Wuant', 'Carlão', 'HMB', 'Yasmine', 'Ray', 'Maria Cecília', 'Xutos', 'Pontapés'])
setP  = set1.union(set2).union(set3).union(set4).union(set5).union(set6).union(set7).union(set10).union(set11).union(set12).union(set13)
print('set1  = ',len(set1))
print('set2  = ',len(set2))
print('set3  = ',len(set3))
print('set4  = ',len(set4))
print('set5  = ',len(set5))
print('set6  = ',len(set6))
print('set7  = ',len(set7))
print('set10 = ',len(set10))
print('set11 = ',len(set11))
print('set12 = ',len(set12))
print('set13 = ',len(set13))
print('setP  = ',len(setP))

set14=set()

print('set14 = ',len(set14))
print('set14 & setP = ',len(set14.intersection(setP)))
print('set14 - setP = ',len(set14.difference(setP)))
toget=list(set14.difference(setP))
print(toget)

set1  =  21
set2  =  35
set3  =  71
set4  =  299
set5  =  276
set6  =  207
set7  =  133
set10 =  966
set11 =  456
set12 =  242
setP  =  2577
set13 =  74
set13 & setP =  0
set13 - setP =  74
['Chris Tamayo', 'Pontapés', 'Furacao Love', 'Mota JR', 'Ella Nor', "Dino d'Santiago", 'Jvst Fly', 'Rafael Allmark', 'Timor YSF', 'Mastiksoul', 'Sea', 'Zara G', 'Domi', 'HMB', 'Apollo G', 'Valas', 'Vado Más Ki Ás', 'Fernanda Salgado', 'xtinto', 'Gson', 'sora9k', 'Wuant', 'MOBBERS', 'WiNDOH', 'Miguel Alves', 'Maria Cecília', 'Seakret', 'Camila Loures', 'Yasmine', 'GROGNation', 'OIK', 'Maycon e Vinicius', 'SYRO', 'Instinto 26', 'Biu do Piseiro', 'Minguito 283', 'Ray Polanco', 'Dream Boyz', 'Mizzy Miles', 'Thascya', 'Junior Lord', 'Aragão', 'LON3R JOHNY', 'Kazzio', 'Ray', 'Huzz', 'Kappa Jotta', 'Achero', 'Yuri NR5', 'Gabriel Elias', 'João Sousa', 'Waze', 'April Ivy', 'Thiago Carvalho', 'Greg Ferreira', 'Yung SpaceGhost', 'Nêgo Jhá', 'Betsy David', 'ÁTOA', 'YUZI', 'Carlão', 'Joint One', 'DJ Pernambuco',

In [ ]:
addAllMusic(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")   ### [Jamie Fine]
addDeezer(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")    ### [Jamie Fine]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")   ### [Elijah Woods]
addDeezer(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")    ### [Elijah Woods]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")   ### [Sueco]
addDeezer(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")    ### [Sueco]
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie]  ,  fast
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie]  ---> [['Sueco']]
### fast  ---> [['Sueco']]


addAllMusic(mdbmaps, "Spotify", "d1b09e1ea18d6a92e651530bbc3755ea", "")   ### [Alicia Moffet]
addDeezer(mdbmaps, "Spotify", "d1b09e1ea18d6a92e651530bbc3755ea", "")    ### [Alicia Moffet]
### Beautiful Scar  ,  Take Control
### Beautiful Scar  ---> [['Alicia Moffet']]
### Take Control  ---> [['Alicia Moffet']]


addAllMusic(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "")   ### [NEW CITY]
addDeezer(mdbmaps, "Spotify", "920c63b4fcad01b381d646d6b8b365bc", "")    ### [NEW CITY]
### Getting Closer
### Getting Closer  ---> [['NEW CITY']]


addAllMusic(mdbmaps, "Spotify", "9ab59de7434b95bb68e6b9f0c5cff97e", "")   ### [Vanic]
addDeezer(mdbmaps, "Spotify", "9ab59de7434b95bb68e6b9f0c5cff97e", "")    ### [Vanic]
### Too Soon (feat. Maty Noyes)
### Too Soon (feat. Maty Noyes)  ---> [['Vanic']]


addAllMusic(mdbmaps, "Spotify", "dc6f8bc1cb6d39457937f5ed10418334", "")   ### [Christian Hudson]
addDeezer(mdbmaps, "Spotify", "dc6f8bc1cb6d39457937f5ed10418334", "")    ### [Christian Hudson]
### Four Leaf Clover - Klowds Remix
### Four Leaf Clover - Klowds Remix  ---> [['Christian Hudson']]


addAllMusic(mdbmaps, "Spotify", "ce5bb3d8056d6d971672242255a6d3b0", "")   ### [Bob Pressner]
addDeezer(mdbmaps, "Spotify", "ce5bb3d8056d6d971672242255a6d3b0", "")    ### [Bob Pressner]
### American Dream
### American Dream  ---> [['Bob Pressner']]


addAllMusic(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "")   ### [DiRTY RADiO]
addDeezer(mdbmaps, "Spotify", "26154d0d5295a4f04f64f0b7f1c0e067", "")    ### [DiRTY RADiO]
### True Love
### True Love  ---> [['DiRTY RADiO']]


addAllMusic(mdbmaps, "Spotify", "180bff77810e1d08186cf125b5d7d353", "")   ### [Cassow]
addDeezer(mdbmaps, "Spotify", "180bff77810e1d08186cf125b5d7d353", "")    ### [Cassow]
### La Brea
### La Brea  ---> [['Cassow']]


addAllMusic(mdbmaps, "Spotify", "c12c928aa06f2f10507af4b3a2c9a934", "")   ### [Madalen Duke]
addDeezer(mdbmaps, "Spotify", "c12c928aa06f2f10507af4b3a2c9a934", "")    ### [Madalen Duke]
### Gucci Store
### Gucci Store  ---> [['Madalen Duke']]


addAllMusic(mdbmaps, "Spotify", "88759173674025cf8f480850e3ba4b0b", "")   ### [Joshua Bassett]
addDeezer(mdbmaps, "Spotify", "88759173674025cf8f480850e3ba4b0b", "")    ### [Joshua Bassett]
### Lie Lie Lie
### Lie Lie Lie  ---> [['Joshua Bassett']]


addAllMusic(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "")   ### [VIC MENSA]
addDeezer(mdbmaps, "Spotify", "2c7704262fe420489299bef485e21f10", "")    ### [VIC MENSA]
### Reverse (Ft. G-Eazy)
### Reverse (Ft. G-Eazy)  ---> [['VIC MENSA']]


addAllMusic(mdbmaps, "Spotify", "2e7143f95646c1272401f710104716dd", "")   ### [DJ Scheme]
addDeezer(mdbmaps, "Spotify", "2e7143f95646c1272401f710104716dd", "")    ### [DJ Scheme]
### Buck 50
### Buck 50  ---> [['DJ Scheme']]


addAllMusic(mdbmaps, "Spotify", "f7e293bc32c1201e5a5a66fa535bf823", "")   ### [Leoni Jut]
addDeezer(mdbmaps, "Spotify", "f7e293bc32c1201e5a5a66fa535bf823", "")    ### [Leoni Jut]
### I'LL BE THERE
### I'LL BE THERE  ---> [['Leoni Jut']]


addAllMusic(mdbmaps, "Spotify", "835daa3012317c49c26a830e14b1e32b", "")   ### [Josh Jips]
addDeezer(mdbmaps, "Spotify", "835daa3012317c49c26a830e14b1e32b", "")    ### [Josh Jips]
### Read
### Read  ---> [['Josh Jips']]


addAllMusic(mdbmaps, "Spotify", "09d950789fd2315b5051dea856a6184f", "")   ### [Skye]
addDeezer(mdbmaps, "Spotify", "09d950789fd2315b5051dea856a6184f", "")    ### [Skye]
### VOICES (feat. XXXTENTACION)
### VOICES (feat. XXXTENTACION)  ---> [['Skye']]


addAllMusic(mdbmaps, "Spotify", "7de9d986161f1cce5810ffdb61615c31", "")   ### [BRELAND]
addDeezer(mdbmaps, "Spotify", "7de9d986161f1cce5810ffdb61615c31", "")    ### [BRELAND]
### My Truck
### My Truck  ---> [['BRELAND']]


addAllMusic(mdbmaps, "Spotify", "7d186492c45514c715a28e500a2f7735", "")   ### [16yrold]
addDeezer(mdbmaps, "Spotify", "7d186492c45514c715a28e500a2f7735", "")    ### [16yrold]
### No Romance
### No Romance  ---> [['16yrold']]


addAllMusic(mdbmaps, "Spotify", "016d228fa16eed569e6887bb0eaa996d", "")   ### [Pedro CapÃ³]
addDeezer(mdbmaps, "Spotify", "016d228fa16eed569e6887bb0eaa996d", "")    ### [Pedro CapÃ³]
### Calma - Remix
### Calma - Remix  ---> [['Pedro Capó', 'Pedro CapÃ³']]


addAllMusic(mdbmaps, "Spotify", "4dc90ffe02b116303dd4830d3f07a656", "")   ### [Rube Claws]
addDeezer(mdbmaps, "Spotify", "4dc90ffe02b116303dd4830d3f07a656", "")    ### [Rube Claws]
### DANG
### DANG  ---> [['Rube Claws']]


mdbmaps["Spotify"].save()

In [35]:
#https://www.hungama.com/albums/lang/russian/popular/
#https://www.kkbox.com/hk/en/artist/FGNJ6VipM2tycJ0F0kas508K-index-1.html
#https://kworb.net/itunes/extended.html
#http://istanbulmusic.blogspot.com/2010/04/best-musicians-from-ecuador.html
manualAppends(toMatch, chartType, minAlbums=0, add=True)

addAllMusic(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")   ### [Jamie Fine]
addDeezer(mdbmaps, "Spotify", "d267166f6f192e4626b58ae273eb6313", "")    ### [Jamie Fine]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")   ### [Elijah Woods]
addDeezer(mdbmaps, "Spotify", "26e4c8109b6b2a3bb8c9f5898b93187f", "")    ### [Elijah Woods]
### Ain't Easy - THE LAUNCH  ,  Catchin Feels
### Ain't Easy - THE LAUNCH  ---> [['Elijah Woods x Jamie Fine']]
### Catchin Feels  ---> [['Elijah Woods x Jamie Fine']]


addAllMusic(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")   ### [Sueco]
addDeezer(mdbmaps, "Spotify", "7f47f9bcd04a663dd4c5b6cc23ce2fc4", "")    ### [Sueco]
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie]  ,  fast
### fast (Remix) [feat. Offset & A Boogie Wit da Hoodie] 

# Extra Artists

## Top40

In [ ]:
#https://www.allmusic.com/artist/boa-mn0000322754
addAllMusic(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "mn0000322754")   ### [Boa]
addDeezer(mdbmaps, "Top40", "6dfb791d61d3dd8a44fa5b5c1c8dbde0", "792")    ### [Boa]
addDeezer(mdbmaps, "Top40", "688f28c172f877020462ae0dcd158f7d", "5337130")    ### [Antytila]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
#https://www.discogs.com/artist/1027379-The-Gospellers
addDiscogs(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "1027379")    ### [Gospellers]
addDeezer(mdbmaps, "Top40", "2874db4bf2285180da2878da30719d98", "7722644")    ### [Gospellers]




addDeezer(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "10470285")    ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "5127ec6e19c0a6008623adfad8b778c8", "72144882")    ### [Moe]
addDeezer(mdbmaps, "Top40", "2bc0fe9eb315a876af7ff1b91988daf3", "4523116")    ### [E.m.m.a.]
addDeezer(mdbmaps, "Top40", "919603bf283bab261d6607bea9234ce3", "75100")    ### [Crashdiet]
addDeezer(mdbmaps, "Top40", "25c0271edadaa16ad11b4016508ad57c", "10540205")    ### [Sebastian]
addDiscogs(mdbmaps, "Top40", "89946a817a31189393f24f92b32be466", "60136")   ### [Catch]
addDeezer(mdbmaps, "Top40", "0b8ff2909d235838583a92b539b640ef", "1164412")    ### [Floodgate]
addDeezer(mdbmaps, "Top40", "680c0282e232b1f64a3ef9d56da97275", "251483")    ### [Wallstones]
addDiscogs(mdbmaps, "Top40", "cb4133f2381241be2ae73d3365e746e8", "3143651")    ### [Honey]
addDeezer(mdbmaps, "Top40", "b59c32c628ceae2c2c6b59aacce5c3fe", "66502")    ### [Nic]
addDeezer(mdbmaps, "Top40", "6269dc570069e12a96dddc0a674e4ff2", "75178")    ### [Charlotte]
addDeezer(mdbmaps, "Top40", "b0e8915fa1946cafcbb743ec3f4a6ea9", "122866")    ### [Charizma]
addDeezer(mdbmaps, "Top40", "afadfe17376050c150bb0f9cb78e4181", "64411")    ### [Fame]
addDeezer(mdbmaps, "Top40", "5984fbaf75a5530e9a3319136ee014cf", "76721")    ### [Dj Roine]
addDiscogs(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "346187")   ### [Jeanette]
addDeezer(mdbmaps, "Top40", "214906164a747a8ce969529a640b9eec", "450752")    ### [Jeanette]
addDeezer(mdbmaps, "Top40", "c652ef0598d18f7655b92b18d87b8bcd", "56449262")    ### [Cuz]
addAllMusic(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "mn0002185241")   ### [H.e.a.t.]
addDiscogs(mdbmaps, "Top40", "89caf52a1eb268d2335ef2f09c39deb1", "1890182")    ### [H.e.a.t.]
addDeezer(mdbmaps, "Top40", "336f4261abc48ea6247320a9b25885b0", "13720517")    ### [Chris Klafford]
addDeezer(mdbmaps, "Top40", "5ae548889336487db92e7746dea26566", "513525")    ### [Jay Smith]
addDeezer(mdbmaps, "Top40", "f412cf34b7f042c6e70017898a977805", "428632")    ### [Jazzattacks]
addDeezer(mdbmaps, "Top40", "71bbc65fd1a58cfe3b7eab94bc666fe4", "4588048")    ### [Ria Hall]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addAllMusic(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "mn0001488122")   ### [J.williams]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "4660f8fef24f1de99d82c58cb6d558ae", "11373")    ### [Solitaire]
addDeezer(mdbmaps, "Top40", "bad689c3aa55f468721a0f0e31e00490", "862972")    ### [Knocks]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDeezer(mdbmaps, "Top40", "ee1825b598532d857e97ba0c03a6c10b", "9049402")    ### [They]
addDeezer(mdbmaps, "Top40", "bdf6a38ef1a824d2fbc71eb0866ec945", "5817516")    ### [Kucka]
addDeezer(mdbmaps, "Top40", "4e61bdac79b1a4b86d16d9a0121b1a77", "1655320")    ### [Tobtok]
addDeezer(mdbmaps, "Top40", "66eaf372536540197d6edb36afe6a9ce", "595")    ### [Destiny`s Child]
addDeezer(mdbmaps, "Top40", "8b3f077c9506a80b91565e354f5939ec", "6756461")    ### [Tony Ansell]
addDeezer(mdbmaps, "Top40", "4dbc5b8f4dabea79e3ae50712cb6a994", "2830461")    ### [Yebba]
addDeezer(mdbmaps, "Top40", "6e9d556d993ea0981f932c68de4c1b77", "9852222")    ### [Kelly Kiara]
addDeezer(mdbmaps, "Top40", "b80ed1fc4530c8fcc8e780deccf26b26", "5358213")    ### [Tom Thum]
addDeezer(mdbmaps, "Top40", "96db126f7d9fb560896964cc9019cfc7", "5922055")    ### [Montaigne]
addDeezer(mdbmaps, "Top40", "ddb7f9b0e8260c58c6f6829d149920a2", "5680005")    ### [James Chatburn]
addDeezer(mdbmaps, "Top40", "7b58a1b910d5311157faf0d7ba943ffa", "6800529")    ### [Benjamin Joseph]
addDeezer(mdbmaps, "Top40", "110294f1b077f24129fc995ebe23fbe8", "6514885")    ### [Beau Ryan]
addDeezer(mdbmaps, "Top40", "620e410c22308e859ec128e1d31cd98f", "323884")    ### [Pez]
addDeezer(mdbmaps, "Top40", "208619acbc541f424ed93ef7efd3f3cd", "6756463")    ### [Peter Wall]
addDeezer(mdbmaps, "Top40", "6b56ba9a8034ebbfd087f70a7f702a1a", "4079487")    ### [Ceekay Jones]
addDeezer(mdbmaps, "Top40", "04d84fa03fde087f3259c8bdc33752bf", "80523")    ### [Tv Rock]
addDeezer(mdbmaps, "Top40", "1be93f066d18ad80f8bfd2e70f363c0b", "1697944")    ### [Mt Raskil Preservation]
addDeezer(mdbmaps, "Top40", "ce801880919906527590fa4ddaa32c45", "1503955")    ### [Musicians Of The Sydney Symphony]
addDiscogs(mdbmaps, "Top40", "f90e3b3a282edb1c217d6e0fcf997d6c", "2931966")   ### [Prophet]
addDeezer(mdbmaps, "Top40", "f31c59cc71f57ef742aa0db40beb9b12", "280294")    ### [B.o.b]
addDeezer(mdbmaps, "Top40", "5f5e50d6614f04aaefd4995dca22f6b0", "443623")    ### [Jes Hudak]
addDeezer(mdbmaps, "Top40", "2176f53302606c207715827ebaa28e2a", "5523244")    ### [George Krikes]
addAllMusic(mdbmaps, "Top40", "1ea871f975022d52aae6eab3e9f86e5e", "mn0000999409")   ### [Geoffrey Gurrumul Yunupingu]
addDeezer(mdbmaps, "Top40", "5b4514a1943a8b0ecfc49c1feb0b95d7", "173576")    ### [R Kelly]
addDeezer(mdbmaps, "Top40", "95f4e2183d8f5105c6f825eae0181a52", "11645017")    ### [Mzwetwo]
addDeezer(mdbmaps, "Top40", "caf2a08a94f016a97e44a10130f3e3cf", "7374704")    ### [G-Easy]
addDeezer(mdbmaps, "Top40", "897216c66805bf28461c18040b04b67b", "10173710")    ### [Esther Elliott]
addAllMusic(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "mn0002135283")   ### [Lz7]
addDeezer(mdbmaps, "Top40", "2f31f75ae3cb6d34dd619a1f3287b9ab", "532751")    ### [Lz7]
addDeezer(mdbmaps, "Top40", "9993a84fddfdfd82804bbfed434d9885", "1825991")    ### [Ruby Frost]
addDeezer(mdbmaps, "Top40", "b242e187ccc3073a56e38ae2ee55e650", "243446")    ### [Drew]
addDeezer(mdbmaps, "Top40", "8e3afdb94e098cd079c307fb95c9218e", "4379877")    ### [Spawnbreezie]
addDeezer(mdbmaps, "Top40", "c8f6f45297ea4a12ea03624a875a212e", "406155")    ### [The Auckland Philharmonia Orchestra]
addDiscogs(mdbmaps, "Top40", "520556e00e6d980c7a428f88a6658c68", "5925200")   ### [Jade Louise]
addDeezer(mdbmaps, "Top40", "2f953cbb100b34dcc4f2d90a8000eacc", "4877604")    ### [Jessie G]
addDeezer(mdbmaps, "Top40", "7b1225826f059f0b7da7a22e128f37fb", "106014482")    ### [Mike Puru]
addDeezer(mdbmaps, "Top40", "79024f1f81e52227802f50f57ad5523d", "1405492")    ### [Seth Haapu]
addDeezer(mdbmaps, "Top40", "ec743dea831270125f51df1df5da8e42", "1182264")    ### [Frisko]
addDiscogs(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "6231462")   ### [Timesthree]
addDeezer(mdbmaps, "Top40", "d5df77c4abe71b7e73ced4bbc963a71f", "1658696")    ### [Timesthree]
addDeezer(mdbmaps, "Top40", "eac8f9698979b290371bd2a392c661fc", "1226318")    ### [Tiki]
addDeezer(mdbmaps, "Top40", "49230df907f220c6fc887f8b142492be", "85563822")    ### [Starboy]
addDeezer(mdbmaps, "Top40", "fca25c35f21674eabb0671a206c03930", "208194")    ### [Kin]
addDeezer(mdbmaps, "Top40", "4b4fd591656c8499d7d511fff78579b7", "11732531")    ### [Pipe Down Project]
addDeezer(mdbmaps, "Top40", "ed0eefd23b87ef67f3d0dd9eadf6226b", "62745")    ### [Arise]
addDeezer(mdbmaps, "Top40", "0e92ee4147f408112ff68c135d5202df", "8915798")    ### [#kiwiscurebatten]
addDeezer(mdbmaps, "Top40", "7f25cab06931e5c754eb74229144ca3a", "1031587")    ### [L. A. B.]
addDiscogs(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "266816")   ### [Jack L]
addDeezer(mdbmaps, "Top40", "26c097ad9f7f55270b85ae02a22bb359", "1438219")    ### [Jack L]
addDeezer(mdbmaps, "Top40", "dabd78a618b6ed0505657303ac766f81", "6788749")    ### [Broken Hill]
addDeezer(mdbmaps, "Top40", "5a73b17da495da0dc6ac4fff5054f0b8", "292468")    ### [Smash Proof]
addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
addAllMusic(mdbmaps, "Top40", "21da687c5162d957e0e8aaa39eff1e80", "mn0003750925")   ### [The Carters]
addAllMusic(mdbmaps, "Top40", "c00c9a5d2e3053716de9b79898cb5691", "mn0001225951")   ### [Mustard]
addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]
addAllMusic(mdbmaps, "Top40", "5a888aa5d5b0835533b2a73dd895b461", "mn0002528230")   ### [DCUP]
addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
addAllMusic(mdbmaps, "Top40", "9d337e01e83fc2b9d9fcbd6930bfb107", "mn0003715465")   ### [Casper Magico]
addAllMusic(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "mn0003496227")   ### [Dakota]
addAllMusic(mdbmaps, "Top40", "29c175308260a766645f88a48fda8cb5", "mn0002141576")   ### [Sasha]
addAllMusic(mdbmaps, "Top40", "20cf770061e851a2768a67afad49a089", "mn0001884165")   ### [Chris Malinchak]
addAllMusic(mdbmaps, "Top40", "80891387fa9995ead4c3a41b8eb58cac", "mn0003165846")   ### [Islove]
addAllMusic(mdbmaps, "Top40", "927d313b814225911b0cc45f6d101384", "mn0003525593")   ### [Neales]
addDiscogs(mdbmaps, "Top40", "000a8119d7146d68b89c0a1190ec7e11", "7474592")    ### [Young T & Bugsey]
addDiscogs(mdbmaps, "Top40", "8869a02f414bf8aa44ffa2491e057c18", "5690424")    ### [Sile Seoige]
addAllMusic(mdbmaps, "Top40", "23766e9564639a78dd2aa4469c656e33", "mn0000186917")   ### [Gabriella]
addAllMusic(mdbmaps, "Top40", "ee06f338805166cc652a5ffa29bfcaa4", "mn0000792831")   ### [Shane Macgowan]
addDiscogs(mdbmaps, "Top40", "338160050dd8760f6e21fd314da0c262", "282163")    ### [Glen Keating]
addDiscogs(mdbmaps, "Top40", "8f7bba2ad693b7bd7a54e9f5561df5fd", "670431")    ### [Redone]
addAllMusic(mdbmaps, "Top40", "61e0cad6ba67b77cf03b21bdcc855000", "mn0001072817")   ### [Ndubz]
addAllMusic(mdbmaps, "Top40", "8c59920da3b9afafbe8064702b8ae417", "mn0000866772")   ### [Levada Louca]
addAllMusic(mdbmaps, "Top40", "f9902337667b007c4cb0bd56853d6f77", "mn0001499382")   ### [Don Mescall]
addDiscogs(mdbmaps, "Top40", "3a78a66700126f076bc9880499d46098", "6754053")    ### [Theresa Rex]
addAllMusic(mdbmaps, "Top40", "45ec0da62cf46f95d1d19d6477e15223", "mn0003492788")   ### [Lewisham & Greenwich NHS Choir]
addAllMusic(mdbmaps, "Top40", "b3a0765a3d6a1f9b5f0218cc1a78167d", "mn0001585547")   ### [DJ Champion]
addAllMusic(mdbmaps, "Top40", "4101fb8a2e7d70d355cb78bb670cafb2", "mn0000615641")   ### [N Trance]
addAllMusic(mdbmaps, "Top40", "d4afc32b214e23fb75edc998fa24f102", "mn0000116049")   ### [D'side]
addAllMusic(mdbmaps, "Top40", "0edfaa56ae9a7023ef89e6dc038b27e7", "mn0000556741")   ### [Marshals]
addDiscogs(mdbmaps, "Top40", "ef5682c651f7c5295bc121ad58f18381", "1627595")    ### [St. Julien]
addAllMusic(mdbmaps, "Top40", "50eab1669ad5105c900ba050d23f7b37", "mn0002085581")   ### [Ruth-anne]
addAllMusic(mdbmaps, "Top40", "123b17da9b4f0bc068b6211be4ba6eeb", "mn0002334760")   ### [Erick]
addDiscogs(mdbmaps, "Top40", "0f4330d2f262dae9ff367c6c07b07ebd", "1551192")    ### [Temper Trap]
addAllMusic(mdbmaps, "Top40", "08de644904d4cf3503bdb90e1095eb0a", "mn0003330733")   ### [High Hopes Choir]
addAllMusic(mdbmaps, "Top40", "1b7dbb15e26894ee42a4088cd348af5c", "mn0002840885")   ### [Frankie Sandford]
addAllMusic(mdbmaps, "Top40", "7eac99ffb3a009020ae826491a7328a2", "mn0002300128")   ### [Calvin Goldspink]
addAllMusic(mdbmaps, "Top40", "a937fec3aae407f22faf4d34080e3c53", "mn0001093965")   ### [Frank Skinner]
addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
addAllMusic(mdbmaps, "Top40", "2bb54463a4b2619f4cddb21626fd43f3", "mn0000443133")   ### [Donaeo]
addDiscogs(mdbmaps, "Top40", "5a56911227b3087c65fa0c64cfc22858", "7175745")    ### [Rani]
addAllMusic(mdbmaps, "Top40", "b44c3d5c561812173cb1fadc6bbadad0", "mn0000663567")   ### [Node]
addAllMusic(mdbmaps, "Top40", "94a91dcb424b476f725b107e3d561dd2", "mn0003085199")   ### [A Typisk]
addDiscogs(mdbmaps, "Top40", "244b104a4ecf5d29056a30135da49164", "5558809")    ### [Amanda]
addDiscogs(mdbmaps, "Top40", "9547f484677db99f07f0d2ce639fbcdb", "2644206")    ### [KESI]
addDiscogs(mdbmaps, "Top40", "cf2ec7a5779606dc225eb1ff52a125b5", "610327")    ### [Six]
addDiscogs(mdbmaps, "Top40", "682ff892fde0720742da7539be5cbfd4", "3769696")    ### [Mellemfingamuzik]
addDiscogs(mdbmaps, "Top40", "964b8471bb5b3317ff80c658789fb5c6", "2178850")    ### [Nadja Ga]
addDiscogs(mdbmaps, "Top40", "5702f798cedf95408e7998cae7160f75", "5094759")    ### [Adhd]
addDiscogs(mdbmaps, "Top40", "5f6a0d2ee2e29501ee8668e87c668653", "8290907")    ### [David O'connor]
addDiscogs(mdbmaps, "Top40", "be31dd84b00112c792101e2ed6bd66b3", "1086751")    ### [Donna & Joe]
addDiscogs(mdbmaps, "Top40", "cec35a4d1e39f4804f0329f8a891b635", "606524")    ### [Johnson]
addDiscogs(mdbmaps, "Top40", "00d39e303b291175cb7dc306b9316607", "1003686")    ### [Kyla]
addAllMusic(mdbmaps, "Top40", "c2a35b01b931ff0ea34c41bb944ed7ad", "mn0000302025")   ### [Rune Rk]
addDiscogs(mdbmaps, "Top40", "906b28e54bbb4b4cf8c91f488d978719", "5024518")    ### [Emile Kruse]
addDiscogs(mdbmaps, "Top40", "b4ac9bf7c517d486d0dfbb31267e4d3f", "6257928")    ### [Pattersutter]
addAllMusic(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "mn0000175401")   ### [M.V.P.]
addDiscogs(mdbmaps, "Top40", "c3eb573746b90008c43fb9ed843e01c4", "396225")    ### [U$o]
addDiscogs(mdbmaps, "Top40", "a5e6e308d31db00836b85dec9a28fac6", "120009")    ### [lum]
addDiscogs(mdbmaps, "Top40", "9cc336f08daf6b60fdc7df51c4951cd7", "771237")    ### [Soren Huss]
addDiscogs(mdbmaps, "Top40", "2f8ef80aa737b08ccbf4a4f597272dd4", "585244")    ### [Jimmy Jorgensen]
addDiscogs(mdbmaps, "Top40", "9361a0dd987d763afb7ebda59733dd4b", "164681")    ### [Michael Andr]
addDiscogs(mdbmaps, "Top40", "8c4759f7357435c07209f893bd031627", "500375")    ### [Goleo Vi]
addDiscogs(mdbmaps, "Top40", "df5925386d5e772c54fab9c8ee4afc1a", "2449670")    ### [Bjornskov]
addDiscogs(mdbmaps, "Top40", "ea6d8ca839c5321096bf52075b5913a3", "1091530")    ### [Blaes Bukki]
addAllMusic(mdbmaps, "Top40", "dff7682d6b7f9e652467f1d387940420", "mn0002116244")   ### [Michael Caro]
addDiscogs(mdbmaps, "Top40", "8293b2b928ccdcfda3a5b282235ed4ba", "489717")    ### [L.o.c.]
addAllMusic(mdbmaps, "Top40", "f101734989e2fb6b8dd4f0192a44424e", "mn0001546498")   ### [Sha]
addDiscogs(mdbmaps, "Top40", "c88659abef342c45aaeee57a7208f24b", "3871612")    ### [Jimillian]
addDiscogs(mdbmaps, "Top40", "cde97ecc04c4fb0ab06efcceb6ba1766", "393367")    ### [Monday]
addDiscogs(mdbmaps, "Top40", "f7a9e3bb639bf960c442d3854cc8e7f7", "709641")    ### [Herrelandsholdet]
addDiscogs(mdbmaps, "Top40", "9c0415d6fbfcac5c210c5cb9cd921ffd", "4181123")    ### [Fruhstuck Bei Stefanie]
addAllMusic(mdbmaps, "Top40", "56fb2c6c60bd0165fcec8d4622f92b80", "mn0002139381")   ### [Buergermeister]
addDiscogs(mdbmaps, "Top40", "bb4ac57918a08f80cb48be770d9e9979", "5134105")    ### [Cinco De Mayo]
addDiscogs(mdbmaps, "Top40", "853397c4ed3d2cd920a67cc9c17067f5", "1883733")    ### [M0]
addAllMusic(mdbmaps, "Top40", "31c4d92d280c3a99ad3e1a44e9adaf5a", "mn0003793253")   ### [Mathias Og Henriette]
addAllMusic(mdbmaps, "Top40", "0d437db44c23416f96cf5975535f221f", "mn0001963267")   ### [B.a.n.g.e.r.s]
addAllMusic(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "mn0002329134")   ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "2c045db8ead27ec90b26be684dee48a8", "100681")    ### [Blaojne]
addDiscogs(mdbmaps, "Top40", "ce330beec86bfeb671dc2a2d08562025", "1943966")    ### [Fresh]
addDiscogs(mdbmaps, "Top40", "396a9170c68a28370c6a415e5cb0acdd", "6119064")    ### [LOTTE]
addDiscogs(mdbmaps, "Top40", "112cbf6ca3a878116b385274aa05438d", "1482382")    ### [RAF]
addDiscogs(mdbmaps, "Top40", "a07c6ac1b552021b88cfd2e02e78ade3", "5072342")    ### [Laura Van Den Elzen]
addDiscogs(mdbmaps, "Top40", "dce6aefe16db0f22d8509a37392fbda0", "34669")    ### [Mia.]
addDiscogs(mdbmaps, "Top40", "a5d240665753e6ec5149a4cfec868fe1", "518132")    ### [Black Fooss]
addDiscogs(mdbmaps, "Top40", "5016655838213c82095f3cd9941ab8a8", "86482")    ### [J-luv]
addDiscogs(mdbmaps, "Top40", "014056a5b370a16da25725557f53497b", "1399728")    ### [Zipfelbuben]
addDiscogs(mdbmaps, "Top40", "2cfa11ab68fa15392a28ba3473bfafd5", "1383823")    ### [Dschungel-allstars 2009]
addDiscogs(mdbmaps, "Top40", "a9d6b7af886dba4eaa94ee0e6a0400f9", "264574")    ### [The Frank and Walters]
addAllMusic(mdbmaps, "Top40", "c74211123c6662f96a81ef1dd607613b", "mn0003687505")   ### [The Greatest Showman Ensemble]
addDiscogs(mdbmaps, "Top40", "977d879a9ef599db0179842a90eb0829", "2986568")    ### [Nik & Ras]
addAllMusic(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "mn0001560738")   ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "506822")    ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "4fb5ad8251f32ebc71a4caa19a79671c", "1404551")    ### [Rald Schmitz]
addDiscogs(mdbmaps, "Top40", "35a5e92b985eb2530625c792eb8794fd", "2261153")    ### [Ell & Nikki]
addDiscogs(mdbmaps, "Top40", "108e43f1076de906cdd8ba014d4b16ff", "2068922")    ### [Bonez Mc]
addDiscogs(mdbmaps, "Top40", "225ac6dbdf0fffd4a9e113da51ca214d", "6081758")    ### [Brudi30]
addDiscogs(mdbmaps, "Top40", "446d29ccb5a2a3de8ac79412defdc3b0", "4255795")    ### [YouNotUs]
addDiscogs(mdbmaps, "Top40", "c3516450970181d37943c2cd92289bfd", "4367398")    ### [Anthony]
addDiscogs(mdbmaps, "Top40", "df31d4c938e84e9804579b48368955ba", "4453381")    ### [Eicka Jane]
addDiscogs(mdbmaps, "Top40", "5921f9df84764cd6c69a4f92a205f390", "7671579")    ### [Lolo]
addAllMusic(mdbmaps, "Top40", "9be0a055a02f9facc0f0fb7fc693c793", "mn0000753347")   ### [Snap!]
addAllMusic(mdbmaps, "Top40", "a0c07f2c1e5e8b60724c92fb83410552", "mn0003380067")   ### [Coleman Hell]
addDeezer(mdbmaps, "Top40", "768af9dd3025f4e218099084284a501b", "54647592")    ### [Mero]
addDeezer(mdbmaps, "Top40", "52752c575988eb5edea512020cec4858", "2211")    ### [Nena]
addDeezer(mdbmaps, "Top40", "e309c2e3fc905eae304b71e10e82eb99", "75530")    ### [Dream]
addDeezer(mdbmaps, "Top40", "8df03772e7cf5b3390e10574a0a90db2", "1407")    ### [Frames]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "1f50464766f3a9ce94f1754b0463ec8f", "182603")    ### [Sarah]
addDeezer(mdbmaps, "Top40", "c3e16affa11432c13e8be7cc6c8d736b", "261212")    ### [XX]
addDeezer(mdbmaps, "Top40", "fbf1e430c96f43c82045103a7c1407fd", "253597")    ### [George Murphy]
addDeezer(mdbmaps, "Top40", "03ece6e9a07c0a5915cfe676a05567a2", "543560")    ### [Kaka]
addAllMusic(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "mn0000084053")   ### [The Jam]
addDeezer(mdbmaps, "Top40", "cdcec0b0558677aa394a85b105a3699c", "3944")    ### [The Jam]
addDeezer(mdbmaps, "Top40", "410637b6027dc82a2ce4a93308e58dca", "90401")    ### [Aura]
addAllMusic(mdbmaps, "Top40", "91f266ac5ec2839f80f07b9c5385c524", "mn0000057556")   ### [Bill & Gloria Gaither]
addDeezer(mdbmaps, "Top40", "856bd94164974e377d780028096bc128", "70715")    ### [Martin]
addDeezer(mdbmaps, "Top40", "f9793851d6cb8247b6b003a729ce9dc6", "264387")    ### [Nash]
addDeezer(mdbmaps, "Top40", "6f21b97f1010287aa78ee44d6046702f", "77424")    ### [Karen]
addDeezer(mdbmaps, "Top40", "50201d87ecab39af8f2f0aabc1cffc24", "166116")    ### [Jasmin]
addDiscogs(mdbmaps, "Top40", "f7c1455d4147cbf4dc094aa9c2a39356", "3594438")    ### [Zoo]
addAllMusic(mdbmaps, "Top40", "35d8d65ac040e8756273c2f76884e5f5", "mn0003842449")   ### [Beast Coast]
addDeezer(mdbmaps, "Top40", "c4d67d31024c80e72927ac50751161f7", "16891")    ### [Nicole]
addDiscogs(mdbmaps, "Top40", "b1b3b97ec0478725400c7c8729607be9", "3037345")    ### [Carol Anthony]
addDeezer(mdbmaps, "Top40", "154e9916e9dc4d52c6dcc9ea8c5934f7", "211032")    ### [Departure]
addDeezer(mdbmaps, "Top40", "a52581e5c20a50979faeecde53d01381", "247134")    ### [Yasmin]
addDeezer(mdbmaps, "Top40", "86058aed9a9a3acf13a9a98a36b7e7b1", "16065")    ### [Louise]
addDeezer(mdbmaps, "Top40", "f4223a2e29ad572bdd62408309749e55", "4068228")    ### [M.A.D.]
addDeezer(mdbmaps, "Top40", "dd0c42047d7063ec53285cda72c0610d", "3244121")    ### [Tough Love]
addDeezer(mdbmaps, "Top40", "e29a4723271638f34847fc292ffa48c3", "271117")    ### [James Fox]
addDeezer(mdbmaps, "Top40", "95b9824e26389cf67674f50cb242ef70", "205804")    ### [Unk]
addAllMusic(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "mn0001453952")   ### [Chima]
addDeezer(mdbmaps, "Top40", "914aff29bbb33d9bd8742b9b8d7bbb7c", "283635")    ### [Chima]
addDiscogs(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "39162")   ### [Clever]
addDeezer(mdbmaps, "Top40", "9545ffea8117bf350955621bae797917", "1261918")    ### [Clever]
addDeezer(mdbmaps, "Top40", "a0eaef106d85a20c4f40697e6be08ade", "1219224")    ### [Rory & The Island]
addDeezer(mdbmaps, "Top40", "7b9aa21b6614e8381a912da660081515", "4791900")    ### [Jordan Egan]
addDeezer(mdbmaps, "Top40", "6ce57bb13155b6a976267ae7fea24a81", "5061058")    ### [The Suspects and Guests]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addDeezer(mdbmaps, "Top40", "331b04fbaf10fec210a976adb4b1d562", "62971")    ### [Heavy]
addDeezer(mdbmaps, "Top40", "4534407ec1d2e60bd79803fcac70ddab", "5613402")    ### [Jimmy Magee]
addDeezer(mdbmaps, "Top40", "93e2f319eb80a4d47ed7830e34472de8", "164956192")    ### [Steffany Gretzinger]
addDeezer(mdbmaps, "Top40", "d027967af998dd89ee301fbfadc2a9c6", "7274")    ### [Down]
addDeezer(mdbmaps, "Top40", "aaa8882482fb1c672c7048f6ad980017", "72652862")    ### [Poppy Fields]
addDeezer(mdbmaps, "Top40", "84ea1e6425431af3e3c9132c3ef0dff2", "5359329")    ### [Presentation Secondary School Clonmel]
addDeezer(mdbmaps, "Top40", "23a4949a6ef2a59b0a1992d55a5e7119", "100307")    ### [Versatile]
addDeezer(mdbmaps, "Top40", "387f8b95bf312f4e1f06c3fa8a2e702d", "7204178")    ### [The Zog Chorus]
addDeezer(mdbmaps, "Top40", "d571e0c4adaf9473e43a55048b524222", "248033")    ### [Chuckie]
addDeezer(mdbmaps, "Top40", "83e36a7fbb4e47d90df2c7a5f41ceca5", "5283366")    ### [Avener]
addDiscogs(mdbmaps, "Top40", "d8af7b562fb5b6896e9caf208a7631fe", "2500803")    ### [Daisy Dares You]
addDeezer(mdbmaps, "Top40", "3771c6784980f14a97a79960df44683f", "7416924")    ### [Emzee A]
addDeezer(mdbmaps, "Top40", "ea418f0376767c59aea193fb88f2fbed", "11807729")    ### [Audrey Trainor]
addDeezer(mdbmaps, "Top40", "f39917252c23494a2bffc34608080e34", "4566327")    ### [Edmee]
addDeezer(mdbmaps, "Top40", "13568fea713fb70f4a72987863820910", "67704502")    ### [Paradise Hotel 2019]
addDeezer(mdbmaps, "Top40", "a552723762e26b8bf2ff3cf00abb742e", "76729")    ### [Vera]
addDeezer(mdbmaps, "Top40", "e20047c96a03c801a7a761d639303baa", "105146")    ### [Christina]
addDeezer(mdbmaps, "Top40", "747e53fad10278c0be9f12e5064e1a8a", "9435020")    ### [Ung Cezar]
addDeezer(mdbmaps, "Top40", "59755826fe21e0d913917f38e519bd97", "4037981")    ### [Trinidad James]
addDiscogs(mdbmaps, "Top40", "b091868e0d9e66c4f3cfc3d57c0e15c1", "6754464")    ### [Pauline]
addDiscogs(mdbmaps, "Top40", "fdef672eb79aa1e42d68559231f9e91a", "2891027")    ### [Contiez]
addDiscogs(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "34061")   ### [DT8]
addDeezer(mdbmaps, "Top40", "376424d25e7c667172ea571a9bd5ecf7", "16653")    ### [DT8]
addDeezer(mdbmaps, "Top40", "298dc80023436fd7d1327755dc0f3aea", "5406531")    ### [Virginia To Vegas]
addAllMusic(mdbmaps, "Top40", "df7a414766c07b227b947fe19d735cca", "mn0000527296")   ### [P.I.]
addDiscogs(mdbmaps, "Top40", "9ffc319f3524de0eb5857b3a041e89d8", "4115550")   ### [Rin]
addDiscogs(mdbmaps, "Top40", "6c757d349b981f59ce15f8050351c33b", "286345")   ### [Androids]
addDiscogs(mdbmaps, "Top40", "2580311e731b9bca913825f23a494d96", "4511617")    ### [Alina]
addDeezer(mdbmaps, "Top40", "f8c6ee3fdec7c8e8d68666db7bd07a5b", "1217914")    ### [Zion & Lennox]
addDeezer(mdbmaps, "Top40", "04b0ea321844b974fa623332ec9e81a8", "1234739")    ### [Nadine]
addDiscogs(mdbmaps, "Top40", "5b813121e727842a70d17d40279c8060", "3915565")    ### [Tieks]
addDeezer(mdbmaps, "Top40", "e6852c19581c5c2842941d6585011de8", "507529")    ### [New Sunset Hotel]
addDeezer(mdbmaps, "Top40", "1ee6a7f79efe291ad8bc9ba50f14ee39", "110592")    ### [Tarja]
addDeezer(mdbmaps, "Top40", "f70f7318448d90d9e2648fa729d0ccd9", "7569860")    ### [Mendoza]
addDeezer(mdbmaps, "Top40", "87019c83d1d9002251d5e0123d73de2f", "4158299")    ### [Ladywell Primary School]
addDeezer(mdbmaps, "Top40", "70fd12ba96ac516bed74bb1918828f96", "7215388")    ### [Wealdstone Raider]
addDeezer(mdbmaps, "Top40", "18a4ef47c6e65a8cd0686f17dcb5a2d6", "458443")    ### [Mike Sheridan]
addDeezer(mdbmaps, "Top40", "b6f0eeb7241d52315a563b41a83f03ab", "12544804")    ### [LIV'n'G]
addDeezer(mdbmaps, "Top40", "3203329df817eab4fd383d2515def0c7", "61557012")    ### [RSPB]
addDeezer(mdbmaps, "Top40", "3de5a67ac001eda9da009b4269f64d72", "84950802")    ### [Tobi & Manny]
addDeezer(mdbmaps, "Top40", "8476f77796a006e9d4cbafa3404aebac", "55142")    ### [Virgo]
addDeezer(mdbmaps, "Top40", "7befb4ecd12fe5e883dc1785be298dab", "134048")    ### [Fabric Bent]
addDiscogs(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "1335584")   ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "9d224a59b3df753402c55e6974c203b5", "152363")    ### [Eddy Bar]
addDeezer(mdbmaps, "Top40", "c039dcafb38ef291cd3576f6f2d96498", "14492701")    ### [Ransom]
addDeezer(mdbmaps, "Top40", "538d641f7c607731ce3c97de84888ddc", "10904")    ### [Trooper]
addAllMusic(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "mn0000043629")   ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "576086654e3dac2a5660989c51366659", "143975")    ### [Bushido Produziert Sonny Black]
addDeezer(mdbmaps, "Top40", "d4e420120a039e03455ee8536740a0a0", "2128")    ### [Goldfinger]
addDeezer(mdbmaps, "Top40", "d9de59336d2d408662aa03ad71e870b1", "4036827")    ### [Andreas Kummert]
addDeezer(mdbmaps, "Top40", "5edc174fb5fefd278c05b53dbd1bd2e0", "14740207")    ### [Prince Karma]
addDeezer(mdbmaps, "Top40", "603304c35dea9d2995f5ea307fc3e437", "5361924")    ### [Judith Van Hel]
addDeezer(mdbmaps, "Top40", "18e08ec319bb5434ce1be10ede35b88d", "15224255")    ### [KMN Gang]
addDeezer(mdbmaps, "Top40", "043c614068c79f134a62512dd92dd205", "4747977")    ### [Meltem Acikgoz]
addDeezer(mdbmaps, "Top40", "f97c6800d97bc9b645de82758b1b616e", "4844456")    ### [Minds Of 99]
addDeezer(mdbmaps, "Top40", "1f84610afa18b37896d9cb8c3ffb929a", "9356090")    ### [POL1Z1STENS0HN]
addAllMusic(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "mn0000748795")   ### [Shifty]
addDeezer(mdbmaps, "Top40", "7dbc379090f7843f537ea0df634b1aad", "6677")    ### [Shifty]
addDeezer(mdbmaps, "Top40", "a17bf82b0c99a5d6a9fdb61e1cba475b", "567150")    ### [Leanne Moore]
addDeezer(mdbmaps, "Top40", "3a9adb2ba05512379660af30c4499d9e", "255246")    ### [Seneca]
addDeezer(mdbmaps, "Top40", "18091df6740eb23606268e1fe70a832e", "262682")    ### [Dustin The Turkey]
addDeezer(mdbmaps, "Top40", "ab0121020ea6f52f5dcc2b0ad8417684", "1569593")    ### [Kanyu Tree]
addDeezer(mdbmaps, "Top40", "1feb0b498fd6b76e794cd4994499bd66", "471918")    ### [Scuba Dice]
addDeezer(mdbmaps, "Top40", "3531e20a4eaab8cd5b085fc9605f6fba", "261093")    ### [Martin Staunton Band]
addDeezer(mdbmaps, "Top40", "8d229bc5952f811627abcecf1a1d9f6d", "404641")    ### [Soul Control]
addAllMusic(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "mn0001951902")   ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "c87cad8f7c42f2e678a0c18782869407", "5441329")    ### [Conway Sisters]
addDeezer(mdbmaps, "Top40", "bf39144a5e6eb48401d5732ec4edddf4", "15811")    ### [Delirious]
addDeezer(mdbmaps, "Top40", "d1029792b9078894873b1965b343540c", "262762")    ### [Walls]
addDiscogs(mdbmaps, "Top40", "009bc268f4161c893c3469d0f475fa03", "393367")    ### [The Revs]
addDeezer(mdbmaps, "Top40", "0166cabfe7594a261af1250ad91c5ebe", "3039")    ### [Di-rect]
addDeezer(mdbmaps, "Top40", "79947bd82f85385cf579852761e3bcca", "5432668")    ### [Grafa]
addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]
addDeezer(mdbmaps, "Top40", "73ea300db5aa1c2154adfe42186a887e", "5418")    ### [Ark]
addDeezer(mdbmaps, "Top40", "c0e4ba1ede3532ccfde5b29e8aacc50b", "789194")    ### [Skryabin]
addDeezer(mdbmaps, "Top40", "12a8a35953b0633264cd439f7dc723ad", "13244969")    ### [Elias]
addDiscogs(mdbmaps, "Top40", "60044c2592b77d0e4a7d4df0d5fdcbba", "908078")    ### [Julia Savicheva]
addDiscogs(mdbmaps, "Top40", "c8c1ade4a56b82b143856c2c590e3fc1", "369368")    ### [Gola]
addDiscogs(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "8707189")   ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8422de1228cdf9aa9ee5888a858a98b1", "5471912")    ### [Mihaela Fileva]
addDeezer(mdbmaps, "Top40", "8e0a54291a8ae6add5c89ae6bd744591", "2077221")    ### [Pavell]
addDeezer(mdbmaps, "Top40", "5793bffe6cf2b42fbcf5ecc4cdf66911", "4796386")    ### [Venci Venc']
addDeezer(mdbmaps, "Top40", "e4a1ed07636cc650fcd92f2ee99a44fe", "1288678")    ### [Lazza]
addDeezer(mdbmaps, "Top40", "c29dd858b42c5e8d6caecd0302c47dfe", "9060182")    ### [Hov 1]
addDeezer(mdbmaps, "Top40", "981871e41d004689023b3867cca2003c", "4239461")    ### [Maan]
addDeezer(mdbmaps, "Top40", "6eb8df7aacfb80b8a5aefbb8155dc13a", "247284")    ### [George Sampanis]
addDeezer(mdbmaps, "Top40", "daf03202751046b2d8cc380e7535cc39", "529")    ### [Raphael]
addDeezer(mdbmaps, "Top40", "8a65b04059e971e76dc736fcf1ab4dd2", "13440379")    ### [Nino]
addDeezer(mdbmaps, "Top40", "1533082f4491090f90aa6f086be1b2b6", "4018571")    ### [Arif]
addDeezer(mdbmaps, "Top40", "3e5169154b281b19469ce1fb806a3919", "795146")    ### [Machete]
addDeezer(mdbmaps, "Top40", "cd5004d3b03507b93a5154c8f703ffb1", "1358419")    ### [KataStrofe]
addDeezer(mdbmaps, "Top40", "f4c09b9cc9b7184e363a22d1a7565ef9", "176639")    ### [Nitro]
addDeezer(mdbmaps, "Top40", "a2f3f9edf03e4bf85df289edf3084780", "58916452")    ### [Dnoteondabeat]
addDeezer(mdbmaps, "Top40", "8bbd146262d01da925cec72df725f3cf", "11937599")    ### [Blockparty]
addDeezer(mdbmaps, "Top40", "0d866c18af54941ec8f33ce3df4be902", "3848")    ### [Elize]
addDeezer(mdbmaps, "Top40", "0c6423fdddb18f976061565d288baded", "1005065")    ### [Como]
addDiscogs(mdbmaps, "Top40", "0f17d781876da4732563e43039aaaa8b", "448654")   ### [L5 (Popstars)]
addDeezer(mdbmaps, "Top40", "ce3fbaee597ec5a524a308b812541f56", "214934")    ### [Rebecca]
addDeezer(mdbmaps, "Top40", "ff69c3ee307619b22c2ee2969aea521d", "11328")    ### [Dj Jose]
addDiscogs(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "2250857")   ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "8f7b584aa940b5b8d46129956fee0c52", "4878214")    ### [Vremia I Steklo]
addDeezer(mdbmaps, "Top40", "1af733e9abe9956b5d2ef9fdbb1ad764", "537741")    ### [Grigoriy Leps]
addDeezer(mdbmaps, "Top40", "a2ee02d0ff400d2ec0d77d2aae27d098", "164423")    ### [Stan]
addDeezer(mdbmaps, "Top40", "cb1a4cdd6178852f36aba64d12602d1a", "281011")    ### [India Martinez]
addDeezer(mdbmaps, "Top40", "2fbb9be1f137d28d5d01a88b2a61c3a6", "1366660")    ### [Taisiya Povaliy]
addDeezer(mdbmaps, "Top40", "43a4a6af3a411e6c9a24170f30c2815b", "13612387")    ### [Maneskin]
addDeezer(mdbmaps, "Top40", "9318518ff509296fc43d312a9d65ff56", "101471")    ### [Pegasus]
addDeezer(mdbmaps, "Top40", "0abaf01f580932db3646af80a6023868", "1710654")    ### [Potap & Nastia]
addAllMusic(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "mn0002080697")   ### [Luttenberger & Klug]
addDeezer(mdbmaps, "Top40", "e3b7acbc2078d13eafa2a765ce38115a", "1495367")    ### [Luttenberger & Klug]
addAllMusic(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "mn0002744178")   ### [Kevin]
addDeezer(mdbmaps, "Top40", "760ffac9c884a2e8fabc13d4eb617e75", "70615702")    ### [Kevin]
addAllMusic(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "mn0003060329")   ### [V. Meladze]
addDeezer(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "4251519")    ### [V. Meladze]
addDiscogs(mdbmaps, "Top40", "c4bfdaef6a75c5b92d4314bed5324ac9", "753179")
addDeezer(mdbmaps, "Top40", "299683e0e48d3508d34c59fd60d1c969", "6138354")    ### [Lora Karadzhova]
addAllMusic(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "mn0002598754")   ### [Tove Ostman Styrke]
addDiscogs(mdbmaps, "Top40", "9277fef69257e9c368980e44addab30d", "2023759")    ### [Tove Ostman Styrke]
addAllMusic(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "mn0002764822")
addDiscogs(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "2241934")   ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "a2b07050b3a3158923a626ecd79a9a99", "1365309")    ### [Maksim Barskih]
addDeezer(mdbmaps, "Top40", "f5cfbe3d9410cd26c9f5db2f5d00e2af", "64976")    ### [Yasin]
addAllMusic(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "mn0001509405")   ### [Valeria]
addDiscogs(mdbmaps, "Top40", "2da7b3559029e16c501f19f2ca9e1f3d", "907332")    ### [Valeria]
addDeezer(mdbmaps, "Top40", "71495276459b2c8d9319a4f23736b1fe", "4188612")    ### [NeAngely]
addDiscogs(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "508430")   ### [Romeo]
addDeezer(mdbmaps, "Top40", "df1a1cf249844e537d5f8e301999f050", "66510")    ### [Romeo]
addDiscogs(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "5163802")   ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "ddcae8e624e066c57c1a76f7a50c3be5", "4444748")    ### [Pianoboy]
addDeezer(mdbmaps, "Top40", "57dd82cd58ebc1ee03495e7111f68837", "69824902")    ### [Adel]
addDeezer(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3097771")   ### [S. Tarabarova]
addDiscogs(mdbmaps, "Top40", "9790bfbf128c234ff3dfe9734a29112d", "3180922")    ### [S. Tarabarova]
addAllMusic(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "mn0000907539")   ### [Dolcenera]
addDeezer(mdbmaps, "Top40", "e87ca03c7907697d6b985a36efec96b3", "77036")    ### [Dolcenera]
addDiscogs(mdbmaps, "Top40", "1a1c4cd2bf4d0da1c3ac37eed6c5e6dd", "349539")   ### [Seer]
addDiscogs(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "323925")   ### [Carola]
addDeezer(mdbmaps, "Top40", "479632c7dad0bc6491b1dc428c2f384b", "9621")    ### [Carola]
addDeezer(mdbmaps, "Top40", "1018fb9997d689d2cfbeb250cba1e0fc", "8731")    ### [Erika]
addAllMusic(mdbmaps, "Top40", "74d19220f1e9035dc9be17ab0fa7d496", "mn0003059139")   ### [Gradysi]
addDeezer(mdbmaps, "Top40", "5eb7994fa8fc5d454eb9975e8d0982b3", "8399")    ### [Wigwam]
addDeezer(mdbmaps, "Top40", "ec7bcd247bd91b161fd3e236faf60113", "8359")    ### [Laura]
addDeezer(mdbmaps, "Top40", "8cc666d82558e9a3801428ce9377e0cc", "243610")    ### [Eri Esittajia]
addDeezer(mdbmaps, "Top40", "14035f38e39640c7b8a126848f15a17f", "6329")    ### [Belinda]
addDeezer(mdbmaps, "Top40", "ace5274eac9ac83ecc8be584f2f89678", "389681")    ### [Jezabels]
addDeezer(mdbmaps, "Top40", "f808aec316082282ad04c543ea7293fc", "1532710")    ### [Gossling]
addDeezer(mdbmaps, "Top40", "8248a0d3d76dd9b6e989ed30758e1fdb", "7727636")    ### [Young Tapz]
addDeezer(mdbmaps, "Top40", "9abfa0a95115150fa9854d05888b30ad", "323")    ### [T-pain]
addDeezer(mdbmaps, "Top40", "9eb2641c9d801ac6a24cd6d40a99cfcc", "1226315")    ### [J.williams]
addDeezer(mdbmaps, "Top40", "d8efa30039a62c84a5cd7aae07baaed4", "9388984")    ### [Jess & Matt]
addDeezer(mdbmaps, "Top40", "3b51465a51f53de3f9406c8c4084b9eb", "4771547")    ### [Hannah]
addDeezer(mdbmaps, "Top40", "ff33e0cca2bc86ca1c8aca2edf6ad878", "146348")    ### [M.V.P.]
addDeezer(mdbmaps, "Top40", "7670c11b0a85e757c3cd25e1cb4f0d2b", "121457")    ### [Dakota]
addDeezer(mdbmaps, "Top40", "aca525c6189a7e1746b237ac3c8e75f1", "241381")    ### [OpShop]
addDeezer(mdbmaps, "Top40", "75addee6118eeea8b15dfa7d27a7bc56", "132842")    ### [Gin]
addDeezer(mdbmaps, "Top40", "40841ab93e4bc05dbd354f23e76fa1ae", "537158")    ### [Sean Og]
addDeezer(mdbmaps, "Top40", "8a8901e987f198d467979ba02675bf2a", "480089")    ### [360]
addDeezer(mdbmaps, "Top40", "8b8ee2fa941154779a27093f715c82c1", "2476")    ### [Whitlams]
addDeezer(mdbmaps, "Top40", "f1e6489254cab6f1fcbe17478f66812d", "9164")    ### [Presets]
addDeezer(mdbmaps, "Top40", "4cbe8a7a881e6bbb1b2ad97de6caa28c", "76000")    ### [Stellar]
addDeezer(mdbmaps, "Top40", "33db69043854dc72c7ae28f79e66b295", "134078")    ### [Bleeders]
addDeezer(mdbmaps, "Top40", "5fbe502afaeea5cd3894fbbacc76d0f0", "72186")    ### [Frontline]
addAllMusic(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "mn0000288296")   ### [k-os]
addDeezer(mdbmaps, "Top40", "8776b0955cc02e70c84df62f5d08783c", "1666")    ### [k-os]
addDeezer(mdbmaps, "Top40", "34110bca84cc003cb7f9663d441b97f3", "404975")    ### [Swiss]
addDeezer(mdbmaps, "Top40", "cf724ae1b389235411e9848ab2492b86", "1263309")    ### [Jason Kerrison]
addDeezer(mdbmaps, "Top40", "92b769e55de140d27e26b4f52382dc3c", "4344203")    ### [JGeek]
addDeezer(mdbmaps, "Top40", "fe6ff35dd5203e256dddc02fa796c5f6", "5611287")    ### [Arise Church]
addDeezer(mdbmaps, "Top40", "b877108ea31ee744be61d4cf33ddf382", "4423398")    ### [Lavina Williams]
addDeezer(mdbmaps, "Top40", "fd41667c368acecbf8d672b7afbed008", "476858")    ### [Aftermath]
addDeezer(mdbmaps, "Top40", "3682bbc007fa90e18375fa5edb0c37b3", "76360")    ### [Toya]
addDeezer(mdbmaps, "Top40", "4157b21887eedb61b04d15ae42183eae", "4757049")    ### [Dolla & Skeet]
addDeezer(mdbmaps, "Top40", "a663ace511580671d69a6b3f916fe919", "1116771")    ### [Catfish & The Bottlemen]
addDiscogs(mdbmaps, "Top40", "dc9afe97092e87a01ec4127299268b14", "97648")    ### [Tone]
addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDeezer(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "7196728")    ### [Jennifer Lawrence]

mdbmaps["Top40"].save()

## BillboardYE

In [ ]:
addDeezer(mdbmaps, "BillboardYE", "52a0965785c941aa958b64dc50f3d972", "176485")    ### [Fuego]
addDeezer(mdbmaps, "BillboardYE", "ccc783c2cd738ace20a5315e7d43e547", "3069")    ### [Rakim]
addDeezer(mdbmaps, "BillboardYE", "edac53ed142896746b3bd4c681c12f64", "347454")    ### [Tim / Berg]
addDeezer(mdbmaps, "BillboardYE", "a7fa9eedf990ab73ba0d7e60db65c95d", "9081")    ### [Eric Prydz Vs. Floyd]
addDeezer(mdbmaps, "BillboardYE", "fbe56cc641e9d04e1b4fcdd924a9c8cc", "318079")    ### [Carlos y Alejandra]

addDeezer(mdbmaps, "BillboardYE", "37f8c0772c31a769f56239563ad7b458", "6500875")    ### [Anuel]
addDeezer(mdbmaps, "BillboardYE", "1c014b1998bb4fab0322f211eec0c611", "445119")    ### [La Adictiva]
addDeezer(mdbmaps, "BillboardYE", "17d5bdd02ad4dee110445a03f919b8ac", "57843")    ### [Marlon]
addDeezer(mdbmaps, "BillboardYE", "33ab8be62d65b7d75df716432b86336f", "240427")    ### [Sky]
addDeezer(mdbmaps, "BillboardYE", "c133daae0111978ba09001d2e7de72f1", "5611")    ### [Willie Colon]
addDeezer(mdbmaps, "BillboardYE", "6510784503b99747dfee77f5a079476d", "50683152")    ### [Banda MS de Sergio Lizarraga
addDeezer(mdbmaps, "BillboardYE", "3f5ee815ed36b113db281ce7ed332da4", "71922")    ### [Dino]
addDeezer(mdbmaps, "BillboardYE", "8046ee19ecf0576f13f6520114447c52", "325722")    ### [James Lloyd]
addDeezer(mdbmaps, "BillboardYE", "73a58c444f3a947331cc0f62fff0cfd5", "628982")    ### [Bastien Laval]
addDeezer(mdbmaps, "BillboardYE", "e28b088f9d859934e4e4cbb39c431199", "4948050")    ### [Il Pomo d'Oro]
addDeezer(mdbmaps, "BillboardYE", "ef95f8a3355a4bbe43fceb8f72be7f9f", "145155")    ### [Divino]
addDeezer(mdbmaps, "BillboardYE", "adc972062945092b4343017780b6c242", "4065944")    ### [Hnos. Vega Jr.]
addDeezer(mdbmaps, "BillboardYE", "8e9f75d573db0b3fd1c5e8b48e3301cc", "4330924")    ### [Codigo FN]

addAllMusic(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "mn0002686289")   ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "0daf21abe59aac57d10e0fe76f304077", "485014")    ### [Ben Williams]
addDeezer(mdbmaps, "BillboardYE", "093b82ff928d96dddbc505cd2f1ddb1f", "469114")    ### [AB Quintanilla]
addDeezer(mdbmaps, "BillboardYE", "d98278206c2f7c2e6232599ee9cb6bc0", "145593")    ### [Adam Holzman]
addDeezer(mdbmaps, "BillboardYE", "dd88591508a27fecc4b05c47f7d9698f", "108626102")    ### [James F. Twyman]

mdbmaps["BillboardYE"].save()

## Billboard

In [ ]:
addDeezer(mdbmaps, "Billboard", "4f617f7ae1994485f0c6fb1e739faa0f", "11064346")    ### [Anibal de Gracia y Sus Invitados]

addDeezer(mdbmaps, "Billboard", "246d14ae520c0bd8adc50cb10538805a", "93555")    ### [Lalo Rodriguez]
addDeezer(mdbmaps, "Billboard", "1f591df9ecc72cdbecc5ca37c953b722", "58203022")    ### [Joe Sambo]
addAllMusic(mdbmaps, "Billboard", "0ac87a2703773df3ee90f35051976d6e", "mn0003205412")   ### [Yuan Yawei]
addDeezer(mdbmaps, "Billboard", "5614aee754da6707437a67aba21ae97b", "9846490")    ### [Gabo Parisi]
addDeezer(mdbmaps, "Billboard", "c751a299bc9b7926f25dd12c5c16c40b", "6459251")    ### [Los de La Nazza]
addDeezer(mdbmaps, "Billboard", "f0ea2c0f6ed64d3d6bc31d1f381092b0", "5138573")    ### [Cantor Yitzchak Meir Helfgot]
addDeezer(mdbmaps, "Billboard", "3c2a14f3f9126d17f7f2fdbfd3e04295", "5162473")    ### [Joe Vasconcelos]
addDeezer(mdbmaps, "Billboard", "fa0c3060dc1ed49905865587788668b7", "4277041")    ### [Hector]
addDeezer(mdbmaps, "Billboard", "d03d6af3ef8c0c4d1e104fe65d94cfb0", "111522992")    ### [Matsuri Nine.]
addDiscogs(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "7964912")    ### [The Elovaters]
addDeezer(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "11723379")    ### [The Elovaters]
addDeezer(mdbmaps, "Billboard", "ae1cfe75bb6f91da7d223452168a3d59", "65634")    ### [Vicious]
addAllMusic(mdbmaps, "Billboard", "1f6381c06850170b54c95e6a596ffb28", "mn0003782313")   ### [Ken Tackey]
addDeezer(mdbmaps, "Billboard", "6b3628493b920913e8d550fd3eb75703", "10908080")    ### [Hoo Leeger]
addDeezer(mdbmaps, "Billboard", "bee91a30c653a9d5146bdce553318cf4", "11975779")    ### [Hwang ChiYeul]
addAllMusic(mdbmaps, "Billboard", "7270cf68e89cab0f5febe2edaed0b7d6", "mn0003487021")   ### [Subaru Shibuya]
addAllMusic(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "mn0002503541")   ### [CNBLUE]
addDeezer(mdbmaps, "Billboard", "42bb5146be764807cc62ea7cd44bff02", "1492698")    ### [CNBLUE]
addDeezer(mdbmaps, "Billboard", "24d5cd50291e2b5c4c2eb9f3cddce277", "65336")    ### [Fiji]
addDeezer(mdbmaps, "Billboard", "725c8f573f9f00fd36ea148172338bbc", "133611")    ### [L'arc En Ciel]
addAllMusic(mdbmaps, "Billboard", "1d5e1f09e81138a1f8efc590f757fc8e", "mn0003255680")   ### [Kakigoori]
addDiscogs(mdbmaps, "Billboard", "a626013386ca6bc138efacdf05aa7a96", "380314")    ### [Mickey]

addDeezer(mdbmaps, "Billboard", "8cf125b4398ec24a60757a1ae0dbde6a", "100731")    ### [Yuri]
addDeezer(mdbmaps, "Billboard", "1a437e23ad6fb8d3077e86589b52a1a3", "9029")    ### [Lucero]
addDeezer(mdbmaps, "Billboard", "a1afb34f24048e019a601fbb42ae6180", "16319")    ### [Banda El Recodo]
addDeezer(mdbmaps, "Billboard", "4b8310b8cd692ffb4a081e8bb4f69ed9", "16599")    ### [Pandora]
addDeezer(mdbmaps, "Billboard", "e75095d73c04898447d4658e618263f3", "1641733")    ### [Jessie Morales: El Original De La Sierra]
addDeezer(mdbmaps, "Billboard", "bff236756e7bf0dd73d59584e3e1bfe7", "4962433")    ### [Rogelio Martinez "EL RM"]
addDeezer(mdbmaps, "Billboard", "c9e0e91d959d73ff5edfb706e6ffa684", "149")    ### [iiO]
addDeezer(mdbmaps, "Billboard", "bb927e3aee770897812deb1e8ab31eae", "264210")    ### [Anaís]
addAllMusic(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "mn0000392577")   ### [R. Ayala]
addDiscogs(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "5845005")    ### [R. Ayala]
addDeezer(mdbmaps, "Billboard", "ddc76b91c35812592603fed613d14f13", "115142")    ### [R. Ayala]

addDeezer(mdbmaps, "Billboard", "878d720ee17c0c0a4ef8f071950f88f6", "17053")    ### [Voyage]
addDeezer(mdbmaps, "Billboard", "d2122c257dd5b8a3c4c0ac06b2abd0e2", "78111")    ### [The Chimes]
addDeezer(mdbmaps, "Billboard", "9ab14919e655dc9669042291d6771569", "7298")    ### [Andy Bell]
addDeezer(mdbmaps, "Billboard", "6c9e35fa812bd11f2496be3aa1076cd2", "1436738")    ### [Kimberly Davis]
addDeezer(mdbmaps, "Billboard", "a4952315ff2618cc4b878de65a4dfe6a", "494171")    ### [Fever]
addDeezer(mdbmaps, "Billboard", "d79329a688889810aa7ee43aef41c59b", "975")    ### [Lime]
addDeezer(mdbmaps, "Billboard", "e8b586570af23bcd228f54f4af667358", "10716495")    ### [Slingshot]
addDeezer(mdbmaps, "Billboard", "37f4a4ef2c5a3db1176b0d514fc5ebf1", "264546")    ### [Bingo Boys]
addDeezer(mdbmaps, "Billboard", "cafd8f2f1480ab5c07e5a677f94539a2", "149")    ### [iio]
addDeezer(mdbmaps, "Billboard", "c858d2c428f54d66c563a498860fbf56", "184317")    ### [Dany]
addDeezer(mdbmaps, "Billboard", "261a823373b1c2d3538e401b824a0705", "120034")    ### [Gia]
addDeezer(mdbmaps, "Billboard", "52aac3cde8dbb07e987225096f7d3df7", "9710684")    ### [GOODBOYS]

addDeezer(mdbmaps, "Billboard", "dd9ca9f1eec3ec7324764098928fa90a", "62490")    ### [Nils]
addDeezer(mdbmaps, "Billboard", "8126b866ffdc150d799057c736ff708e", "577628")    ### [Steve Oliver]
addDeezer(mdbmaps, "Billboard", "4eb35c41efdc8495ebca5e64cb723217", "4518873")    ### [Jeff Ryan]
addDeezer(mdbmaps, "Billboard", "96aa7eddeddf7079189c5f9c2feb0378", "1594391")    ### [Kim Scott]

addDeezer(mdbmaps, "Billboard", "a0ec3ffe08120ccf0c9b5345714a5c51", "2743")    ### [LTD]
addDiscogs(mdbmaps, "Billboard", "fa00044058206494d4335094632fe4eb", "83751")    ### [Boss]
addDeezer(mdbmaps, "Billboard", "9a0fdd63c4aa97fc9716413a6111c187", "148184")    ### [Enchantment]
addDeezer(mdbmaps, "Billboard", "ef9d8e82a6937e87668db5702cd43b2f", "1816361")    ### [UMC's]
addDeezer(mdbmaps, "Billboard", "71e904485de1ca65917eb99f886f9c4d", "69608322")    ### [Marty]
addDeezer(mdbmaps, "Billboard", "7a85a9d8804e19413bab514f5f51dfb9", "8322164")    ### [Donnie Trumpet & The Social Experiment]
addDeezer(mdbmaps, "Billboard", "4fdc8908f2ee7ad25e35f2ad9531c681", "15080705")    ### [Dennis Reed & Gap]
addDeezer(mdbmaps, "Billboard", "6c999baddd68939e501aa0561d3e0eda", "4052173")    ### [King Bach]
addDeezer(mdbmaps, "Billboard", "ba6cb719ca53bf4f57ecdcd6bd342bf5", "59927")    ### [Magoo]
addDeezer(mdbmaps, "Billboard", "55be138b7b3faad5764a62dc13c9c402", "1236346")    ### [John Reilly]
addDeezer(mdbmaps, "Billboard", "4ba36e510dd7bbd2009fb8ff90aa48c7", "1575744")    ### [Francis & The Lights]
addDeezer(mdbmaps, "Billboard", "07c0deba90ed751e5c2ec92d0d60d7dd", "59218332")    ### [Brooke Staten]

addDeezer(mdbmaps, "Billboard", "66dca5283eac29a7b8c9714ed919bb78", "442981")    ### [Red Sun Rising]
addDeezer(mdbmaps, "Billboard", "7087547bca580c898106d8f7eb761f58", "251366")    ### [Michael Peterson]
addDeezer(mdbmaps, "Billboard", "b470feb6f3d72bfae87a83e16dd83886", "76640")    ### [Sons of the Desert]

addDeezer(mdbmaps, "Billboard", "a6fbf98a79a048834d50ce68d67ad442", "546")    ### [Mya]
addDeezer(mdbmaps, "Billboard", "3bb7b26b6dee914d5d35651092ebbd38", "123941")    ### [Questian Mark & The Mysterians]
addDeezer(mdbmaps, "Billboard", "c5281bc610aaf5fcb1be3ea9b5aef4a7", "1078918")    ### [Bo Donaldson And The Heywoods]
addDeezer(mdbmaps, "Billboard", "ee476cb24bebb0cbe4d7061b0d33a707", "1533511")    ### [Rock Star Supernova]
addDeezer(mdbmaps, "Billboard", "31aa5ad81b2a6342572b6bd54425deae", "390649")    ### [Iration]
addDeezer(mdbmaps, "Billboard", "808c8b3a9909c7d7a3e5753792d1b995", "1225601")    ### [LV]

mdbmaps["Billboard"].save()

In [ ]:
updateManDB(2)

In [ ]:
toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)

for idx,row in toMatchDF.iterrows():
    artistName = row["ArtistName"]
    albumNames = row["ArtistAlbums"]
    nAlbums    = row["NumAlbums"]
    if nAlbums >= 1:
        print(artistName,'\t',nAlbums)
    else:
        break
#https://kworb.net/itunes/extended.html



# Renames

In [ ]:
cad.getArtistData("Dream")

In [ ]:
toget, togetID, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)
analyzeRenames(manDB, dbRenames)

In [ ]:
updateManDB(2)

In [ ]:
togetID

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsDiscogs import dbArtistsDiscogs
dbAP = dbArtistsAllMusic()




artistURLs=["https://www.allmusic.com/artist/heat-mn0002185241"]
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)
    
dbAP = dbArtistsDiscogs()
artistURLs=['https://www.discogs.com/artist/60136-Catch', 'https://www.discogs.com/artist/3143651-Honey-31', 'https://www.discogs.com/artist/346187-Jeanette-Biedermann']
for artistURL in artistURLs:
    artistID=dbAP.dutils.getArtistID(artistURL)
    savename=dbAP.dutils.getArtistSavename(artistID)
    dbAP.dutils.downloadArtistURL(artistURL, savename)

#Artist ID [0003255680] is not a member of artistIDToName.
#Artist ID [0002503541] is not a member of artistIDToName.
#Artist ID [0003487021] is not a member of artistIDToName.
#Artist ID [7964912] is not a member of artistIDToName.
#Artist ID [0003205412] is not a member of artistIDToName.

In [ ]:
from dbArtistsDeezer import dbArtistsDeezer
from time import sleep
dbAP = dbArtistsDeezer()
print(togetID.keys())
for db in togetID.keys():
    if db != "Deezer":
        continue
    for dbID,artistName in togetID["Deezer"].items():
        artistURL = dbAP.getArtistURL(dbID)
        artistSavename = dbAP.dutils.getArtistSavename(dbID)
        try:
            dbAP.dutils.downloadArtistURL(artistURL, artistSavename, force=False, sleeptime=2)
        except:
            sleep(2)

In [ ]:
updateManDB(4)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
updateManDB(4)

# Suffix

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
cad.getAlbumsData("George Frideric Handel: Messiah")

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(2)

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
updateManDB(2)

In [ ]:
cad.getAlbumsData("Common Ground")

In [ ]:
updateManDB(2)

# Analyze Matches

In [ ]:
1/0
addAllMusic(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "mn0000417717")   ### [Raybion Bros.]
addDiscogs(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "2188194")    ### [Raybion Bros.]
### Butterfly Kisses

addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
### My Dogs

addAllMusic(mdbmaps, "Top40", "866a64f5a5e684f61aa573af7f63bc57", "mn0000233423")   ### [Magoo]
### Up Jumps Da Boogie

addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
### Hold Me

addAllMusic(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "mn0000186401")   ### [Karen O.]
addDiscogs(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "245778")    ### [Karen O.]
### Hello Tomorrow

addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]

addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
### White Walls

addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDiscogs(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "4210193")    ### [Jennifer Lawrence]
### The Hanging Tree

addAllMusic(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "mn0003323773")   ### [Lookas]
addDiscogs(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "3619379")    ### [Lookas]
### GDFR

addAllMusic(mdbmaps, "Top40", "f37bd04269e23d0c4c10c435a4f1dd7f", "mn0002943184")   ### [Mnek]
### Never Forget You  ,  So Good

addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
### I Hate U, I Love U  ,  Lights Down Low

addAllMusic(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "mn0000351749")   ### [Royce Da 5'9]
addDiscogs(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "40792")    ### [Royce Da 5'9]
### Not Alike

addAllMusic(mdbmaps, "Top40", "296504ec4590a1522dcf6afb838fd6f4", "mn0003332851")   ### [London On Da Track]
### Numbers

mdbmaps["Top40"].save()

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(4)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch["Top40"]])

In [ ]:
saveData = []
for fullName in sorted(list(cad.manyArtists.keys())):
    appendName = " & ".join(["[{0}]".format(manDB.renamed(indivName)) for indivName in sorted(list(cad.manyArtists[fullName]))])
    saveData.append([fullName, {"Guess": appendName, "Truth": ''}]) # = list(cad.manyArtists[fullName].keys())
saveFile(idata=saveData, ifile="tmp.yaml")

In [ ]:
fixedData = getFile("tmp.yaml")

In [ ]:
multis = []
multirenames = {}
for item in fixedData:
    if len(item[1]["Truth"]) > 0:
        name = item[1]["Truth"]
        if len(name.split("] & [")) == 1:
            multis.append(item[1]["Truth"][1:-1])
        else:
            rename = name[1:-1].split("] & [")
            multirenames[item[0]] = " ::: ".join(rename)
        
print("Found {0} renames".format(len(multirenames)))
print("Found {0} multi-artists".format(len(multis)))

#### Save MultiArtists (if found)

In [ ]:
knownFilename = "../multiartist/knownMultiArtists.yaml"
knownMultis = getFile(knownFilename)
print(len(multis))
print(len(knownMultis))
knownMultis += multis
print(len(knownMultis))
knownMultis = sorted(list(set(knownMultis)))
print(len(knownMultis))
saveFile(idata=knownMultis, ifile=knownFilename)

#### Save multi renames (if needed)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
#multiManDB.forceReload(getFile("main.yaml"))
multiManDB.addRenames(multirenames)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
multiManDB.addRenames(multirenames)
multiManDB.save()
saveFile(idata=multiManDB.getRenames(), ifile="multi.yaml")
multiManDB = masterArtistNameDB("multi", init=True)
multiManDB.forceReload(getFile("multi.yaml"))
multiManDB.checkForRecursives()
multiManDB = masterArtistNameDB("multi", init=False)

In [ ]:
#### Edit main.yaml (if needed)

In [ ]:
mdbmaps["Billboard"].getDF()

In [ ]:
mdf = mdbmaps["BillboardYE"].getDF()
mdf.shape
#mdf[mdf["DBMatches"] == 0]

# Multi Match

In [ ]:
cad.getTypeArtistAlbumData("Joe Rene")

In [ ]:
toMatch["Billboard"]

In [ ]:
manualAppends(cad, mType, toMatch, chartType, minAlbums=0, add=True)

In [ ]:
mdbmc.getMasterDF("Billboard")

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.85), toMatch=toMatch, useAlbums=False)
reMatch()

In [ ]:
#singleArtistAlbumData["Billboard"]['Ernest']
saveFile(idata=[x[1]["ArtistName"] for x in toMatch["Top40"]], ifile="toget.p")

# Find Near Artist Name Matches

In [ ]:
toget, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)

In [ ]:
if len(dbRenames) > 0:
    analyzeRenames(manDB, dbRenames)
    # Test for overap with manDB

    dbRenames = getFile(ifile="relDBRenames2.yaml")
    redos = {}
    dels  = []
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys(): # and bestname not in manDB.artistNameDB.keys():
            #print("Older=[{0}] \t Best=[{1}]".format(oldername,bestname))

            redos[bestname] = oldername
            dels.append(oldername)
            #print('PROBLEM ({0} in master DB): \t{1}  -->  {2}'.format(oldername, oldername, bestname))
            try:
                matchOlderData = mdbmaps[chartType].getArtistDataByName(oldername).getDict()
            except:
                matchOlderData = {}

            try:
                matchBestData  = mdbmaps[chartType].getArtistDataByName(bestname).getDict()
            except:
                matchBestData = {}

            #print("Older: {0}".format(matchOlderData))
            #print("Best:  {0}".format(matchBestData))
            #print("\n")
        else:
            print("{0}: {1}".format(oldername, bestname))

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()
manDB.save()
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
vals="""
[Sander Van Doom                0.9                Sander Van Doorn] 	 --> (Top40) 27813b44b4282de50171e13adb4ebcf9 / 183267 (Discogs)
"""

lines=vals.split("\n")
lines2=[x.split("({0}) ".format(chartType))[1] for x in lines if len(x) > 0]
names2=[x.split("({0}) ".format(chartType))[0] for x in lines if len(x) > 0]
lines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}
names3=[x.split("    ")[0].strip()[1:] for x in names2]
if len(names3) != len(lines3):
    raise ValueError("Err")

for i,(k,v) in enumerate(lines3.items()):
    artistName = names3[i]
    #print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', \'{2}\')".format(k,v,))
    print("### ----> {0}".format(artistName))
    print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', None)".format(k,v))    
    albums = singleArtistAlbumData[chartType][artistName]
    print("### {0}".format("  ,  ".join(albums)))
    print("### ignores.append(\"{0}\")".format(artistName))
    print("\n")
    
    
#mdbmaps[chartType].addArtistDataByID("e309c2e3fc905eae304b71e10e82eb99", "AllMusic", '0000803469')
print("")
print("mdbmaps[chartType].save()")
print("saveFile(idata=list(set(ignores)), ifile=\"chartIgnores.yaml\")")

In [ ]:
### ----> Sander Van Doom
mdbmaps[chartType].addArtistDataByID('27813b44b4282de50171e13adb4ebcf9', 'Discogs', None)
### Grasshopper
### ignores.append("Sander Van Doom")



mdbmaps[chartType].save()
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

In [ ]:
toget

****
****
****
****

# Multis

In [ ]:
ignores.append('Daryl Hall & John Oates')

In [ ]:
from multiArtist import multiartist
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
print(len(knownMultiArtists))
print(len(mdbmaps[chartType].getArtists()))
for idx,artistName in mdbmaps[chartType].getArtists().items():
    N = mularts.getArtistNames(artistName)
    if len(N) > 1:
        result = {subName: mdbmaps[chartType].isKnownByName(subName) for subName in N}
        if not any(result.values()):
            knownMultiArtists.append(artistName)
            #print(result)
        #print("ignores.append(\'{0}\')".format(artistName))

print(len(knownMultiArtists))
knownMultiArtists = list(set(knownMultiArtists))
print(len(knownMultiArtists))
        
#saveFile(idata = sorted(knownMultiArtists), ifile="../multiartist/knownMultiArtists.yaml")

In [ ]:
ignores = getFile("chartIgnores.yaml")
dels = []
for idx,artistName in mdbmaps[chartType].getArtists().items():
    if artistName in ignores:
        dels.append([idx,artistName])
        
print(len(dels))
for idx,artistName in dels:
    mdbmaps[chartType].removeArtistByID(idx)
    
mdbmaps[chartType].save()

In [ ]:
len(dels)

# Functions

In [ ]:
class matchthresholds:
    def __init__(self):
        thresholds = {}
        thresholds[1000] = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 10.0}
        thresholds[500]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 5.0}
        thresholds[200]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 2.5}
        thresholds[100]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(100/8), 'score': 1.5}
        thresholds[50]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(50/8), 'score': 1.5}
        thresholds[20]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 3, 'score': 1.5}
        thresholds[10]   = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[5]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[3]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[2]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 2.0}
        thresholds[1]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 0.9}
        self.thresholds = thresholds

        iterItems = {20: {"Max": 50, "Vals": 100}}
        iterItems.update({10: {"Max": 20, "Vals": 200}})
        iterItems.update({50: {"Max": 10000, "Vals": 50}})
        iterItems.update({5: {"Max": 10, "Vals": 250}})
        iterItems.update({3: {"Max": 5, "Vals": 500}})
        iterItems.update({2: {"Max": 3, "Vals": 500}})
        iterItems.update({1: {"Max": 2, "Vals": 500}})
        self.iterItems = iterItems

        self.thresholdsNoAlbums = {'numArtistName': 5, 'artistNameCutoff': 0.9, 'artistAlbumCutoff': None, 'numArtistAlbums': None, 'score': None}
    
    def getThresholdsWithAlbums(self, minAlbums):
        if self.thresholds.get(minAlbums) is not None:
            return self.thresholds[minAlbums]
        
    def getThresholdsWithoutAlbums(self, cutoff):
        self.thresholdsNoAlbums["artistNameCutoff"] = cutoff
        return self.thresholdsNoAlbums
    
    def getIterItems(self):
        return self.iterItems
        

In [ ]:
from functools import partial
from matchDBArtist import matchDBArtist

class multimatch:
    def __init__(self, maindb, mdbmc, debug=False):
        self.debug  = debug
        self.maindb = maindb
        self.mdbmc  = mdbmc
        
        self.mt = matchthresholds()
        
        self.toMatch = None
        self.dbName  = None
        self.maxMatches = None
        
        self.matchFunc = {True: self.matchDBArtistWithAlbums, False: self.matchDBArtistWithoutAlbums}
            
        
    def setMDBMap(self, dbName, mdbmap):
        self.mdbmap = mdbmap
        self.dbName = dbName
    
    
    def setDataToMatch(self, toMatch):
        if toMatch is not None:
            self.toMatch = toMatch
            
            
    def setMaxMatches(self, maxMatches):
        self.maxMatches = maxMatches
        
        
    ############################################################################################################
    ## Set Data To Be Matched
    ############################################################################################################
    def setData(self, useAlbums=True, minAlbums=None, maxValues=None, cutoff=None):
        self.useAlbums = useAlbums
        print("HI")
        if useAlbums is True:
            if minAlbums is not None:
                self.thresholds = self.mt.getThresholdsWithAlbums(minAlbums)
                iterItems = self.mt.getIterItems()[minAlbums]
                maxAlbums = iterItems["Max"]
                if maxValues is None:
                    maxValues = thresholds["Vals"]

                self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=[])
                print("Found {0} Artists To Match With Albums and Using Thresholds: {1}".format(len(self.toMatch[self.dbName]), self.thresholds))
            else:
                raise ValueError("Must supply a minAlbums value")
        else:
            self.thresholds = self.mt.getThresholdsWithoutAlbums(cutoff)
            if maxValues is None:
                maxValues = 100000
            self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=100000, minAlbums=0, ignores=[])
            print("Found {0} Artists To Match Without Albums and Without Using Thresholds".format(len(self.toMatch[self.dbName])))
            
    
    


    def matchItAll(self):
        num_processes = 3
        func       = self.matchFunc[self.useAlbums]
        thresholds = self.thresholds
        pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
        argument_list = self.toMatch[self.dbName]
        print("Args: ",argument_list)
        
        if len(argument_list) == 0:
            return
        print("Using {0} Matching Function".format(pfunc))
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = self.multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)
        return result_list
        

    def matchDBArtistWithoutAlbums(self, item, *args, **kwargs):
        return 1

        print("Item ==>",item)
        1/0
        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = None
        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score']),
        mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def matchDBArtistWithAlbums(self, item, *args, **kwargs):    
        #time.sleep(0.0025)

        return 1
        print("Item ==>",item)
        1/0
        

        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = artistData["ArtistAlbums"]

        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score'])
        mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def multiProc(self, func, argument_list, num_processes):
        pool = Pool(processes=num_processes)
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
            result_list_tqdm.append(result)
        return result_list_tqdm


In [ ]:
mm = multimatch(maindb, mdbmc)
mm.setMDBMap("Billboard", mdbmaps["Billboard"])

In [ ]:
mm.setData(useAlbums=True, minAlbums=10, maxValues=1)

In [ ]:
mm.matchItAll()

In [ ]:
ignoresList = ["chartIgnores.yaml"]
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
        
reMatch()
print("Done.\n\n")

In [ ]:
from billboardCharts import billboardCharts
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
from os.path import join
from searchUtils import findExt
import urllib
from time import sleep
from collections import Counter
from artistIgnores import getArtistIgnores

from billboardCharts import billboardCharts
from top40Charts import top40Charts

class billboardFiles:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Billoard"

    def findFiles(self):
        savedir = join(self.basedir, "data", "billboard", "results")
        self.files   = findExt(savedir, ext='.p')
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
    
    
class billboardFullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Billboard"
        self.charts  = charts
        self.tfiles  = billboardFiles(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))
        
    
        
    def setFullChartData(self):
        fullChartData = {}
        renameStats   = Counter()
        
        self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)


class chartData:
    def __init__(self, source, minYear=None, maxYear=None, country=None, debug=False):
        debug=False    
        self.source = source
        
        self.basedir  = "/Volumes/Piggy/Charts/"
        self.basename = source
        

        try:
            self.cts    = {"Billboard": billboardCharts(), "Top40": top40Charts()}[source]
            self.cFiles = {"Billboard": billboard40files(self.basedir), "Top40": top40files(self.basedir)}[source]
            self.files  = self.cFiles.findFiles()
        except:
            raise ValueError("Could not create charts data for {0}".format(source))

                        
        self.charts = []
            
        self.minYear   = minYear
        self.maxYear   = maxYear
        
        self.artistRenames   = {}
        self.dbRenames       = {}

        self.chartData       = {}
        self.fullChartData   = {}
        self.artistAlbumData = {}
        
        
    #####################################################################################################################################
    ## Artist Data
    #####################################################################################################################################
    def getArtists(self):
        return list(self.artistAlbumData.keys())
        
        
    
    #####################################################################################################################################
    ## Full Chart Data
    #####################################################################################################################################
    def getFullChartDataFilename(self):
        ifile="current{0}FullChartArtistAlbumData.p".format(self.basename)
        return ifile

    def getFullChartData(self):
        return getFile(self.getFullChartDataFilename())
        
    def saveFullChartData(self):
        print("Saving {0} Full Artist Data".format(len(self.fullChartData)))
        saveFile(idata=self.fullChartData, ifile=self.getFullChartDataFilename(), debug=True)        
        
        
    
    #####################################################################################################################################
    ## Artist Album Data
    #####################################################################################################################################
    def getArtistAlbumDataFilename(self):
        ifile="current{0}ArtistAlbumData.p".format(self.basename)
        return ifile
    
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        
    
    def getArtistAlbumData(self):
        return getFile(self.getArtistAlbumDataFilename())
        
    def saveArtistAlbumData(self):
        print("Saving {0} Artist Album Data to {1}".format(len(self.artistAlbumData), self.getArtistAlbumDataFilename()))
        saveFile(idata=self.artistAlbumData, ifile=self.getArtistAlbumDataFilename(), debug=True)  
        
        
    
    #####################################################################################################################################
    ## Rename Data
    #####################################################################################################################################        
    def setRenames(self, artistRenames):
        self.artistRenames = artistRenames
        
    def setDBRenames(self, dbRenames):
        self.dbRenames = dbRenames
        
        
    
    #####################################################################################################################################
    ## Find Files
    #####################################################################################################################################
    def setChartUsage(self, name=None, rank=None):
        if rank is not None:
            if isinstance(rank, list):
                for item in rank:
                    self.charts += self.cts.getChartsByRank(item)
            elif isinstance(rank, int):
                self.charts += self.cts.getChartsByRank(rank)
        elif name is not None:
            self.charts += self.cts.getCharts(name)
        else:
            self.charts = self.cts.getCharts(None)
        if name is None:
            name = "None"
        print("  Using Charts ({0}): {1}".format(name, self.charts))
        
        
        
                
    #####################################################################################################################################
    ## Set Artist Album Data
    #####################################################################################################################################
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        print("There are {0} unique artist entries".format(len(self.artistAlbumData)))
        

    #####################################################################################################################################
    ## Set Full Chart Data
    #####################################################################################################################################
    def setFullChartData(self):
        {"Billboard": self.setFullChartDataBillboard(), "Top40": self.setFullChartDataTop40()}[self.source]
    
    def setFullChartDataBillboard(self):        
        renameStats  = Counter()
        chartCounter = Counter()
        
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
                
                for date, dResults in cnameResults.items():
                    if self.minYear is not None:
                        if getDateTime(date).year < int(self.minYear):
                            continue
                    if self.maxYear is not None:
                        if getDateTime(date).year > int(self.maxYear):
                            continue
                    stryear = getDateTime(date).year

                    artist = dResults["Artist"]

                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist 
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist  

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue


                    chartCounter[chartName] += 1

                    album  = dResults["Name"]

                    if self.chartData.get(artist) is None:
                        self.chartData[artist] = Counter()
                    self.chartData[artist][album] += 1
                    
                    if self.fullChartData.get(artist) is None:
                        self.fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if self.fullChartData[artist][key].get(album) is None:
                        self.fullChartData[artist][key][album] = {}
                    if self.fullChartData[artist][key][album].get(chartName) is None:
                        self.fullChartData[artist][key][album][chartName] = {}
                    self.fullChartData[artist][key][album][chartName][date] = 0
                #print("{0: <40}{1}".format("{0}-{1}".format(chartName,stryear),len(self.fullChartData)))
                
        
    def setFullChartDataTop40(self):
        fullChartData = {}
        renameStats   = Counter()
        
        #self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)